# Readme before you Run

1. This will take a while to run. It brings in some files from the web, which are slow to access. Also, the National Categories file is slow to load
2. This code uses some libraries that you, the user, may not have installed. Those are **geopandas** & **ipywidgets**. Please make sure you have them installed before you run the script.
3. Please use the latest version of Jupyter Notebook (Version: 7.0.6). Otherwise, I am not sure it will all run and present itself as I intended.
4. I had to suppress warnings because when I updated my notebook, I started getting error that i checked are a bug
5. Hope you enjoy the analysis 

In [1]:
# pip install geopandas
# pip install ipywidgets
# conda update --all

# Code:

In [2]:
import warnings

def function_that_warns():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    function_that_warns()  # this will not show a warning


import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import requests
import json
import numpy as np
import ipywidgets as widgets
import time
import scipy as sp
import statsmodels.api as sm

## Bring in the data

### Heavy Data

In [4]:
# Function that helps with analsys
def Info(df):
    print(list(df.columns))
    print(print(df.info()))
    display(df.head())

In [5]:
# Import the FTE File
url = 'https://files.digital.nhs.uk/41/2B0F5C/GPW%20Regional%20Tables%20-%20September%202023.xlsx'
staff = pd.read_excel(url, sheet_name='Data for 3a & 3b')
df_staff = staff.copy()

In [6]:
# Import the Patients File
url = 'https://files.digital.nhs.uk/41/2B0F5C/GPW%20Regional%20Tables%20-%20September%202023.xlsx'
patients = pd.read_excel(url, sheet_name='2')
df_pat = patients.copy()

In [7]:
# Region
url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/NHS_England_Regions_July_2022_EN_BFC_2022/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(url)
print(response.status_code)
print(response.headers['Date'])
regions = gpd.read_file(url)
df_regions = regions.copy()

200
Thu, 07 Dec 2023 08:48:23 GMT


In [8]:
# ICB
url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Integrated_Care_Boards_April_2023_EN_BSC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(url)
print(response.status_code)
print(response.headers['Date'])
ICBs = gpd.read_file(url)
df_ICBs = ICBs.copy()

200
Thu, 07 Dec 2023 08:49:10 GMT


In [9]:
# sub-ICB
url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Sub_Integrated_Care_Board_Locations_April_2023_EN_BFC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(url)
print(response.status_code)
print(response.headers['Date'])
SubICBs = gpd.read_file(url)
df_SubICBs = SubICBs.copy()


200
Thu, 07 Dec 2023 08:49:42 GMT


In [10]:
df_SubICBs = SubICBs.copy()

In [11]:
df_nc = pd.read_excel('national_categories.xlsx')
df_nc_holder = df_nc.copy()


In [12]:
url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/SICBL22_ICB22_NHSER22_EN_LU/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'
response = requests.get(url)
print(response.status_code)
print(response.headers['Date'])
districts = gpd.read_file(url)

200
Thu, 07 Dec 2023 08:52:35 GMT


## Functions

In [13]:
# Name Cleaning Dictionary Work

# This needs to be re-run for updating. Should not delete 
# df_dict = pd.read_csv('col2name.csv') 
# display(df_dict)
# col2namedict = df_dict.set_index("Column")["Name"].to_dict()
# name2coldict = df_dict.set_index("Name")["Column"].to_dict()

col2namedict = {'All Patients': 'Number of Patients', 'appointment_mode': 'Appointment Mode', 'appointment_month': 'Appointment Month', 'appointment_status': 'Appointment Status', 'Ave_apt_attened (AD) per Admin': 'Avg. Monthly Attended / # Admin', 'Ave_apt_attened (AD) per FTE': 'Avg. Monthly Attended / # FTE', 'Ave_apt_attened (AD) per GP': 'Avg. Monthly Attended / # GP', 'Ave_apt_long (NC) per Admin': 'Avg. Monthly Scheduled (NC) / # Admin', 'Ave_apt_long (NC) per FTE': 'Avg. Monthly Scheduled (NC) / # All FTEs', 'Ave_apt_long (NC) per GP': 'Avg. Monthly Scheduled (NC) / # GP', 'Ave_apt_recent (AR) per Admin': 'Avg. Monthly Scheduled (AR) / # Admin', 'Ave_apt_recent (AR) per FTE': 'Avg. Monthly Scheduled (AR) / # All FTEs', 'Ave_apt_recent (AR) per GP': 'Avg. Monthly Scheduled (AR) / # GP', 'average_delay': 'Ave. Time to Appointment (days)', 'average_duration': 'Ave. Duration of Appointment ', 'count_of_appointments': 'Total # of Appointments', 'count_of_appointments_ave_ad': 'Avg. Monthly Attended', 'count_of_appointments_ave_ar': 'Avg. Monthly Scheduled (AR)', 'count_of_appointments_ave_nc': 'Avg. Monthly Scheduled (NC)', 'FTE%:Admin/Non-Clinical': '%age Admin FTEs', 'FTE%:Direct Patient Care': '%age Dir. Care FTEs', 'FTE%:GP': '%age GPs', 'FTE%:Nurses': '%age Nurses', 'FTE%:Total': 'Dropdfsd', 'FTE:Total': 'Total FTEs', 'hcp_type': 'HCP Type', 'ICB_Name': 'ICB Name', 'Pat%:15 and below': '%age of 15 & Below', 'Pat%:15-44': '%age of 14-44y Olds', 'Pat%:45-64': '%age of 45-65y Olds', 'Pat%:65 and over': '%age of 65 & Over', 'Pat:0-4': 'Patients below 4', 'Pat:15 and below per Admin': 'Patient (15 & Below)', 'Pat:15 and below per FTE': 'Patient (15 & Below) Per FTE', 'Pat:15 and below per GP': 'Patient (15 & Below) per GP', 'Pat:15-44': 'Patients (15-44y)', 'Pat:15-44 per Admin': 'Patient (15-44y) per Admin FTE', 'Pat:15-44 per FTE': 'Patient (15-44y) per FTE', 'Pat:15-44 per GP': 'Patient (15-44y) per GP', 'Pat:45-64': 'Patients (45-65y)', 'Pat:45-64 per Admin': 'Patient (45-65y) per Admin FTE', 'Pat:45-64 per FTE': 'Patient (45-65y) per FTE', 'Pat:45-64 per GP': 'Patient (45-65y) per GP', 'Pat:65 and over': 'Patients (65 & Over)', 'Pat:65 and over per Admin': 'Patient (65 & Over) per Admin FTE', 'Pat:65 and over per FTE': 'Patient (65 & Over) per FTE', 'Pat:65 and over per GP': 'Patient (65 & Over) per GP', 'Pat:85 and over': 'Patient (85 & Over)', 'Patients per FTE': 'Patient per FTE', 'Patients per GP': 'Patient per GP', 'Region_Full_Name': 'Region Full Name', 'time_between_book_and_appointment': 'Time Between Book & Appointment', 'icb_ons_code': 'ICB Code', 'count_of_appointments_ave_ar per Pats': 'Avg. Monthly Scheduled (AR) / # Patients', 'count_of_appointments_ave_ad per Pats': 'Avg. Monthly Attended (AD) / # Patients', 'count_of_appointments_ave_nc per Pats': 'Avg. Monthly Scheduled (NC) / # Patients', 'sub_icb_location_ons_code': 'Sub ICB Location Ons Code', 'sub_icb_short_code': 'Sub ICB Short Code', 'Sub_ICB_Location_name': 'Sub ICB Location Name', 'appointment_date': 'Appointment Date', 'service_setting': 'Service Setting', 'context_type': 'Context Type', 'national_category': 'National Category', 'weekday': 'Weekday', 'service_setting*': 'Service Setting*', 'context_type*': 'Context Type*', 'national_category*': 'National Category*'}
name2coldict = {'Number of Patients': 'All Patients', 'Appointment Mode': 'appointment_mode', 'Appointment Month': 'appointment_month', 'Appointment Status': 'appointment_status', 'Avg. Monthly Attended / # Admin': 'Ave_apt_attened (AD) per Admin', 'Avg. Monthly Attended / # FTE': 'Ave_apt_attened (AD) per FTE', 'Avg. Monthly Attended / # GP': 'Ave_apt_attened (AD) per GP', 'Avg. Monthly Scheduled (NC) / # Admin': 'Ave_apt_long (NC) per Admin', 'Avg. Monthly Scheduled (NC) / # All FTEs': 'Ave_apt_long (NC) per FTE', 'Avg. Monthly Scheduled (NC) / # GP': 'Ave_apt_long (NC) per GP', 'Avg. Monthly Scheduled (AR) / # Admin': 'Ave_apt_recent (AR) per Admin', 'Avg. Monthly Scheduled (AR) / # All FTEs': 'Ave_apt_recent (AR) per FTE', 'Avg. Monthly Scheduled (AR) / # GP': 'Ave_apt_recent (AR) per GP', 'Ave. Time to Appointment (days)': 'average_delay', 'Ave. Duration of Appointment ': 'average_duration', 'Total # of Appointments': 'count_of_appointments', 'Avg. Monthly Attended': 'count_of_appointments_ave_ad', 'Avg. Monthly Scheduled (AR)': 'count_of_appointments_ave_ar', 'Avg. Monthly Scheduled (NC)': 'count_of_appointments_ave_nc', '%age Admin FTEs': 'FTE%:Admin/Non-Clinical', '%age Dir. Care FTEs': 'FTE%:Direct Patient Care', '%age GPs': 'FTE%:GP', '%age Nurses': 'FTE%:Nurses', 'Dropdfsd': 'FTE%:Total', 'Total FTEs': 'FTE:Total', 'HCP Type': 'hcp_type', 'ICB Name': 'ICB_Name', '%age of 15 & Below': 'Pat%:15 and below', '%age of 14-44y Olds': 'Pat%:15-44', '%age of 45-65y Olds': 'Pat%:45-64', '%age of 65 & Over': 'Pat%:65 and over', 'Patients below 4': 'Pat:0-4', 'Patient (15 & Below)': 'Pat:15 and below per Admin', 'Patient (15 & Below) Per FTE': 'Pat:15 and below per FTE', 'Patient (15 & Below) per GP': 'Pat:15 and below per GP', 'Patients (15-44y)': 'Pat:15-44', 'Patient (15-44y) per Admin FTE': 'Pat:15-44 per Admin', 'Patient (15-44y) per FTE': 'Pat:15-44 per FTE', 'Patient (15-44y) per GP': 'Pat:15-44 per GP', 'Patients (45-65y)': 'Pat:45-64', 'Patient (45-65y) per Admin FTE': 'Pat:45-64 per Admin', 'Patient (45-65y) per FTE': 'Pat:45-64 per FTE', 'Patient (45-65y) per GP': 'Pat:45-64 per GP', 'Patients (65 & Over)': 'Pat:65 and over', 'Patient (65 & Over) per Admin FTE': 'Pat:65 and over per Admin', 'Patient (65 & Over) per FTE': 'Pat:65 and over per FTE', 'Patient (65 & Over) per GP': 'Pat:65 and over per GP', 'Patient (85 & Over)': 'Pat:85 and over', 'Patient per FTE': 'Patients per FTE', 'Patient per GP': 'Patients per GP', 'Region Full Name': 'Region_Full_Name', 'Time Between Book & Appointment': 'time_between_book_and_appointment', 'ICB Code': 'icb_ons_code', 'Avg. Monthly Scheduled (AR) / # Patients': 'count_of_appointments_ave_ar per Pats', 'Avg. Monthly Attended (AD) / # Patients': 'count_of_appointments_ave_ad per Pats', 'Avg. Monthly Scheduled (NC) / # Patients': 'count_of_appointments_ave_nc per Pats', 'Sub ICB Location Ons Code': 'sub_icb_location_ons_code', 'Sub ICB Short Code': 'sub_icb_short_code', 'Sub ICB Location Name': 'Sub_ICB_Location_name', 'Appointment Date': 'appointment_date', 'Service Setting': 'service_setting', 'Context Type': 'context_type', 'National Category': 'national_category', 'Weekday': 'weekday', 'Service Setting*': 'service_setting*', 'Context Type*': 'context_type*', 'National Category*': 'national_category*'}


def col2name(fcolname):
    if fcolname in col2namedict:
        out = col2namedict.get(fcolname)
    else:
        out = fcolname
    return out

def name2col(fnamename):
    if fnamename in name2coldict:
        out = name2coldict.get(fnamename)
    else:
        out = fnamename
    return out




In [14]:
sns.set(rc={"figure.figsize": (10, 5)})
sns.set_style('white')
    
to_delete = ['i', 'myvar','make_sumbarplot_local','chart1','drop_down','make_interaction','xlist','xlist1','xlist2',
            'myvar','app1','app2','app3','app4','item1','item2','item3','item4','check1','check2','check3','check4',
            'check5','wlist','ddwig1','ddwig2']

# Generic function to make a chart 
def make_sumbarplot_global(heading):
    total=df.count_of_appointments.sum()
    data4chart1 = df.groupby(heading)['count_of_appointments'].agg('sum').to_frame()
    data4chart1.rename_axis(None,axis=0,inplace=True)
    li = list(data4chart1.sort_values(by=data4chart1.columns[0],ascending=False).index)
    chart1 = sns.barplot(x=data4chart1.index, 
                              y='count_of_appointments', 
                              data=data4chart1,
                              order=li,
                              estimator='sum',
                              palette='coolwarm')

    sns.despine(left=True, bottom=False)  
    chart1.tick_params(labelrotation=90)
    barplotlabels2(chart1,total)
    return chart1

# Generic function to put labels on a bar chart
def barplotlabelsper(chart2):
    for rect in chart2.patches:
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_y() + rect.get_height()
        label = "{:,.1f}%".format(rect.get_height()*100)
        chart2.annotate(label,(x + 0.05, y), rotation = 45) 

# Generic function that turns DataFrame into series of lists 
def make_lists(fdf,fcol_list):
    fwlist = []
    for i in fcol_list:
        randx = list(fdf[i].unique())
        randx.insert(0,'All')
        
        fix_list = ['Inconsistent Mapping', 'Unmapped']
        for j in fix_list:
            if j in randx:
                randx.remove(j)
                randx.append(j)
        fwlist.append(randx)

    return fwlist

In [15]:
# Variation of a function that makes labels on charts. it uses size of the point to format it accordicly.
def barplotlabels(chart2):
    for rect in chart2.patches:
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_y() + rect.get_height()
        if y > 10000000:
            label = "{:,.1f}m".format(rect.get_height()/1000000)
            chart2.annotate(label,(x + 0.05, y), rotation = 45) 
        elif y > 1000000:
            label = "{:,.2f}m".format(rect.get_height()/1000000)
            chart2.annotate(label,(x + 0.05, y), rotation = 45) 
        elif y > 500000:
            label = "{:,.0f}K".format(rect.get_height()/1000)
            chart2.annotate(label,(x + 0.05, y), rotation = 45) 
        elif y > 100000:
            label = "{:,.1f}K".format(rect.get_height()/1000)
            chart2.annotate(label,(x + 0.05, y), rotation = 45) 
        elif y > 10000:
            label = "{:,.2f}K".format(rect.get_height()/1000)
            chart2.annotate(label,(x + 0.05, y), rotation = 45) 
        elif y > 1:
            label = "{:,.2f}".format(rect.get_height())
            chart2.annotate(label,(x + 0.05, y), rotation = 45) 
        else:
            label = "{:,.0f}".format(rect.get_height())
            chart2.annotate(label,(x + 0.05, y), rotation = 45)

# Variation of a function that makes labels on charts. It uses the size of the point to format it accordingly.
def barplotlabels2(chart2, total):
    for rect in chart2.patches:
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_y() + rect.get_height()
        pctstr = "{:,.1f}%".format(y/total*100)
        if y > 10000000:
            label = "{:,.1f}m".format(rect.get_height()/1000000)+" ("+pctstr+")"
        elif y > 1000000:
            label = "{:,.2f}m".format(rect.get_height()/1000000)+" ("+pctstr+")"
        elif y > 500000:
            label = "{:,.0f}K".format(rect.get_height()/1000)+" ("+pctstr+")"
        elif y > 100000:
            label = "{:,.1f}K".format(rect.get_height()/1000)+" ("+pctstr+")"
        elif y > 10000:
            label = "{:,.2f}K".format(rect.get_height()/1000)+" ("+pctstr+")"
        else:
            label = "{:,.0f}".format(rect.get_height())+" ("+pctstr+")"
        if y/total > 0.005:
            chart2.annotate(label,(x + 0.05, y), rotation = 45)

In [16]:
# Variation of a function that makes labels on charts. It uses the size of the point to format it accordingly.
def barplotlabels2per(fchart, total1, total2):
    i = 0
    for rect in fchart.patches:
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_y() + rect.get_height()/2
        pctstr1 = "{:,.1f}%".format(rect.get_height()/total2[i]*100)
        pctstr2 = "{:,.1f}%".format(rect.get_height()/total1*100)
        pctstrT = " ("+pctstr1+")"
        #         print("{:,.0f}".format(y), pctstr, total)

        if y > 10000000:
            label = "{:,.1f}m".format(rect.get_height()/1000000)+ pctstrT
        elif y > 1000000:
            label = "{:,.2f}m".format(rect.get_height()/1000000)+ pctstrT
        elif y > 500000:
            label = "{:,.0f}K".format(rect.get_height()/1000)+ pctstrT
        elif y > 100000:
            label = "{:,.1f}K".format(rect.get_height()/1000)+ pctstrT
        elif y > 10000:
            label = "{:,.2f}K".format(rect.get_height()/1000)+ pctstrT
        else:
            label = "{:,.0f}".format(rect.get_height())+ pctstrT
        if rect.get_height()/total1 > 0.005:
            fchart.annotate(label ,(x-0.1 , y), rotation = 45)
        i += 1
        if i == len(total2): i = 0 

# " ("+pctstr1+',' +pctstr2+")"

In [17]:
# Variation of a function that makes labels on charts. It uses the size of the point to format it accordingly.

def barplotlabels3(fchart, total):
    for rect in fchart.patches:
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_y() + rect.get_height()/2
        pctstr = "{:,.1f}%".format(rect.get_height()/total*100)
#         print("{:,.0f}".format(y), pctstr, total)
        if y > 10000000:
            label = "{:,.1f}m".format(rect.get_height()/1000000)+" ("+pctstr+")"
        elif y > 1000000:
            label = "{:,.2f}m".format(rect.get_height()/1000000)+" ("+pctstr+")"
        elif y > 500000:
            label = "{:,.0f}K".format(rect.get_height()/1000)+" ("+pctstr+")"
        elif y > 100000:
            label = "{:,.1f}K".format(rect.get_height()/1000)+" ("+pctstr+")"
        elif y > 10000:
            label = "{:,.2f}K".format(rect.get_height()/1000)+" ("+pctstr+")"
        else:
            label = "{:,.0f}".format(rect.get_height())+" ("+pctstr+")"
        if rect.get_height()/total > 0.005:
            fchart.annotate(label,(x-0.1 , y), rotation = 45)

# Variation of a function that makes labels on charts.
def barplotlabels4(fchart, total):
    for rect in fchart.patches:
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_y() + rect.get_height()/2
        pctstr = "{:,.1f}% of g.avg".format(rect.get_height()/total*100)
        label = "{:,.2f}".format(rect.get_height())+" ("+pctstr+")"
        if rect.get_height()/total > 0.005:
            fchart.annotate(label,(x-0.1 , y), rotation = 90)


In [18]:
# A function that gets a dataframe but exlcudes a list of categories from up to two variables.

def get_twoway_df(fdf,fheading, fhuein=[], fexclude1 = [], fexclude2 = []):
    data4chart1 = fdf

    if fexclude1 != [] and fexclude2 != []:
        # print('yes,yes')
        exclude_filter = ~data4chart1[fheading].str.contains('|'.join(fexclude1), case=True) 
        data4chart1 = data4chart1[exclude_filter]
        exclude_filter = ~data4chart1[fhuein].str.contains('|'.join(fexclude2), case=True) 
        data4chart1 = data4chart1[exclude_filter]
    elif fexclude1 != [] and fexclude2 == []: 
        # print('yes,no')
        exclude_filter = ~data4chart1[fheading].str.contains('|'.join(fexclude1), case=True) 
        data4chart1 = data4chart1[exclude_filter]
    elif fexclude1 == [] and fexclude2 != []:
        # print('yes,no')
        exclude_filter = ~data4chart1[fhuein].str.contains('|'.join(fexclude2), case=True) 
        data4chart1 = data4chart1[exclude_filter]
    else:
        # print('no,no')
        data4chart1 = fdf    
    return data4chart1

In [19]:
# Function that sorts a data series in a particular way, with an option to sort a specific column in a specific way
# This one focuses on count of appointments
def sort_data_coa(fdata4chart1,fheading, fhuein, fexclude1, fexclude2):
    fdata4chart1 = fdata4chart1.groupby([fheading,fhuein])['count_of_appointments'].agg('sum').reset_index()
    tempdata = fdata4chart1.groupby(fheading)['count_of_appointments'].agg('sum').reset_index()
    
    if fheading == 'time_between_book_and_appointment':
        li = ['Same Day','1 Day', '2 to 7 Days', '8  to 14 Days',
              '15  to 21 Days', '22  to 28 Days', 'More than 28 Days', 
              'Unknown / Data Quality']
    else:
        
        tempdata = tempdata.sort_values(by='count_of_appointments',ascending=False)
        li = list(tempdata[fheading])
   
    fdata4chart1[fheading] = pd.Categorical(fdata4chart1[fheading], categories=li)
    
# Function that sorts a data series in a particular way, with an option to sort a specific column in a specific way
# This one focuses on average delay
def sort_data_avdel(fdata4chart1,fheading,):
    tempdata = fdata4chart1.groupby(fheading)['ave_delay'].agg('mean').reset_index()

    if fheading == 'time_between_book_and_appointment':
        li = ['Same Day','1 Day', '2 to 7 Days', '8  to 14 Days',
              '15  to 21 Days', '22  to 28 Days', 'More than 28 Days', 
              'Unknown / Data Quality']
    else:
        li = list(tempdata.sort_values(by='ave_delay',ascending=False)[fheading])
   
    fdata4chart1[fheading] = pd.Categorical(fdata4chart1[fheading], categories=li)    

In [20]:
# Another version of the sorting function
def sort_data_4chart(fdata4chart1,fheading,fincol,ftype):
    tempdata = fdata4chart1.groupby(fheading)[fincol].agg(ftype).reset_index()
    
    if fheading == 'time_between_book_and_appointment':
        li = ['Same Day','1 Day', '2 to 7 Days', '8  to 14 Days',
              '15  to 21 Days', '22  to 28 Days', 'More than 28 Days', 
              'Unknown / Data Quality']
    else:
        li = list(tempdata.sort_values(by=fincol,ascending=False)[fheading])
   
    fdata4chart1[fheading] = pd.Categorical(fdata4chart1[fheading], categories=li)

In [21]:
# Another version of the sorting function
def sort_data_4charttest(fdata4chart1,fheading,fincol,ftype):
    print(fheading,fincol,ftype)
    tempdata = fdata4chart1.groupby(fheading).agg(ftype).reset_index()[fincol]
    
    display(tempdata.head())
    
    if fheading == 'time_between_book_and_appointment':
        li = ['Same Day','1 Day', '2 to 7 Days', '8  to 14 Days',
              '15  to 21 Days', '22  to 28 Days', 'More than 28 Days', 
              'Unknown / Data Quality']
    else:
        li = list(tempdata.sort_values(by=fincol,ascending=False)[fheading])
   
    fdata4chart1[fheading] = pd.Categorical(fdata4chart1[fheading], categories=li)
    return fdata4chart1



In [22]:
# Gets a two-way dataset for a ratio of numerical variables grouped by two categorical variables, with exclusions but not yet implemented 
def get_twoway_data_avdel(fdf,fheading, fhuein=[], fexclude1=[], fexclude2=[], ftype='sum',fincol='delay_atot_float'):
    coa ='count_of_appointments'
    
    df_temp = fdf
    df_temp = df_temp[[fheading,coa,fincol]].dropna() if fhuein==[] else df_temp[[fheading,fhuein,coa,fincol]].dropna()
    temp_dict = {coa:ftype,fincol:ftype}
    if fhuein==[]:
        df_temp = df_temp.groupby([fheading])[[fincol, coa]].agg(temp_dict).reset_index()
    else:
        df_temp = df_temp.groupby([fheading,fhuein])[[fincol, coa]].agg(temp_dict).reset_index()
    
    df_temp['ave_delay'] = df_temp[fincol] / df_temp[coa]
    return df_temp



In [23]:
# Gets a two-way dataset for a numerical variable grouped by two categorical variables, with exclusions but not yet implemented 
def get_twoway_data_coa(fdf,fheading, fhuein=[], fexclude1=[], fexclude2=[], ftype='sum',fincol='count_of_appointments'):
    df_temp = fdf
    # print('inside data call',fheading, fhuein, fexclude1, fexclude2) 
   

#     df_temp[('hcp_type')].sum()

    df_temp = df_temp[[fheading,fincol]].dropna() if fhuein==[] else df_temp[[fheading,fhuein,fincol]].dropna()


    if fhuein==[]:
        # print('no')
        df_temp = df_temp.groupby([fheading])[fincol].agg(ftype).reset_index()
    else:
        # print('yes')
        df_temp = df_temp.groupby([fheading,fhuein])[fincol].agg(ftype).reset_index()
    
    return df_temp

In [24]:
# This function gives a list of column headings of a two column level dataframe
def makelistfromtupels(fwlist):
    outlist = []
    outdict = {}
    for i in range(len(fwlist)):
        a = [j for j in fwlist[i]]
#         print(a[0]+', '+a[1])
        value = a[0]+' w/ '+a[1] if a[1]!='' else a[0]
#         print(value)
        outlist.append(value)
        outdict[value] = i 

    #         outdict[value] = fwlist[i] 
    return outlist, outdict


In [25]:
# Generic format of a chart 
def format_chart1(fchart1):
    sns.color_palette("coolwarm", as_cmap=True)
    fchart1.tick_params(labelrotation=90)
    sns.despine(left=True, bottom=False)     
    sns.set_style('white')
    return fchart1

In [26]:
# Three way groupby, that also has an option to have 'All' as an input.
def choose_data_three_Appoint(fdf,fcol_list,x_value,item1, item2, item3):
    method = 'sum'
    data4chart1 = fdf[[fcol_list[0],fcol_list[1],fcol_list[2], 'count_of_appointments', x_value]]
    if item1 == 'All':
        if item2 == 'All':
            if item3 == 'All': # All are All
                data4chart1 = data4chart1.groupby([x_value])['count_of_appointments'].agg(method).to_frame()
            else: # NC is not All
                data4chart1 = data4chart1[(data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame()
        else: # CT is not ALL
            if item3 == 'All': # CT is not ALL
                data4chart1 = data4chart1[(data4chart1[fcol_list[1]]==item2)].groupby([x_value])['count_of_appointments'].agg(method).to_frame()
            else: # CT is not ALL & NC is not All
                data4chart1 = data4chart1[(data4chart1[fcol_list[1]]==item2) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame()
    else: # SS are not All
        if item2 == 'All': # SS are not All
            if item3 == 'All': # SS are not All
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1)].groupby([x_value])['count_of_appointments'].agg(method).to_frame()
            else: # SS are not All & NC is not ALL
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame()
        else: # SS are not All & & CT is not ALL
            if item3 == 'All': # SS in not ALL & CT is not ALL
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[1]]==item2)].groupby([x_value])['count_of_appointments'].agg(method).to_frame()
            else: # SS in not ALL & CT is not ALL & NC is not All
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[1]]==item2) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame()

    data4chart1.columns = ['count_of_appointments']

    return data4chart1

In [27]:
# Function that puts circles around points on a scatter graph
def make_scatter_labels(ffdatain,ffxdata,ffydata,fflabel):
    ffdatain.reset_index(inplace=True)
    
    for i in range(len(ffdatain[ffxdata])):
        x = ffdatain[ffxdata][i]
        y = ffdatain[ffydata][i]
        label = ffdatain[fflabel][i]
        bbox = {'boxstyle': 'circle, pad=0.5', 'fill': False, 'edgecolor':'maroon'}
        plt.annotate(' ', (x, y)  , rotation = 45, ha='center', va='center',size='small',bbox=bbox) 

        plt.text(x, y*1.04, label , rotation = 45, ha='center', va='center',size='small') 


In [28]:
# Untilty function for choosing right inputs based on type of geospatial analysis being performed
def get_geo_df(fgeoswitch):    
    if fgeoswitch == "R":
        fdfgeo = df_regions.copy()
    elif fgeoswitch == "M":
        fdfgeo = df_ICBs.copy()
    elif fgeoswitch == "S":
        fdfgeo = df_SubICBs.copy()
    return fdfgeo
    
def get_right_map_data(fgeoswitch):    
    if fgeoswitch == "R":
        fdataout =df_mapR
    elif fgeoswitch == "M":
        fdataout = df_mapM
    elif fgeoswitch == "S":
        fdataout = df_mapS
    return fdataout

def get_code(fgeoswitch):
    if fgeoswitch == "R":
        fregvar = ['region_ons_code']
    elif fgeoswitch == "M":
        fregvar = ['icb_ons_code']
    elif fgeoswitch == "S":
        fregvar = ['sub_icb_location_ons_code']        
    return fregvar        

In [29]:
# Puts labels on a map
def maplabels1(fswitch,fitem1,fdfgeo=None):
    # returns 
    fitem1 = name2col(fitem1)
   
    if fdfgeo is None:
        loc4map1 = get_geo_df(fswitch)
    else:
       loc4map1 = fdfgeo
       loc4map1.reset_index(inplace = True)

    data4map1 = get_right_map_data(fswitch)
    code_col = get_code(fswitch)
    data_dict = data4map1[code_col+[fitem1]].set_index(code_col)[fitem1].to_dict()    

    # print(code2nam_dict_RR)
    
    for i in range(len(loc4map1)):
        x = loc4map1['LONG'][i]
        y = loc4map1['LAT'][i]
        geo_code = loc4map1[code_col[0]][i]
        num = data_dict.get(geo_code)
   
        if fswitch == "R": textlab = code2nam_dict_RR.get(geo_code) 
        if fswitch == "M": textlab = code2nam_dict_MM.get(geo_code) 

        textlab = textlab + ':' +'\n'
        
        # pctstr = "{:,.1f}%".format(num/total*100)
        # pctstr = f" ({pctstr})"
        # print(x,y, geo_code)
        # print(textlab,geo_code)
        # print(num)

        if num > 10000000:
            label = textlab + "{:,.1f}m".format(num/1000000)
        elif num > 1000000:
            label = textlab + "{:,.2f}m".format(num/1000000)
        elif num > 500000:
            label = textlab + "{:,.0f}K".format(num/1000)
        elif num > 100000:
            label = textlab + "{:,.1f}K".format(num/1000)
        elif num > 10000:
            label = textlab + "{:,.2f}K".format(num/1000)
        elif num > 1000:
            label = textlab + "{:,.0f}".format(num)
        elif num > 100:
            label = textlab + "{:,.0f}".format(num)
        elif num > 20:
            label = textlab + "{:,.0f}".format(num)
        elif num > 10:
            label = textlab + "{:,.1f}".format(num)
        elif num > 1:
            label = textlab + "{:,.2f}".format(num)
        elif num > 0.5:
            label = textlab + "{:,.2f}".format(num)
        elif num > 0.05:
            label = textlab + "{:,.3f}".format(num)
        else:
            label = textlab + "{:,.2f}".format(num)

        # print(label)
        exeptions = ['London'] 
    
        if textlab in exeptions:
            if fswitch == "R": 
                plt.text(x, y, label , rotation = 0, ha='center', va='center',size='large') 
            else:
                plt.text(x, y, label , rotation = 0, ha='center', va='center',size='small') 
        else:
            if fswitch == "R": 
                plt.text(x, y, label , rotation = 0, ha='center', va='center',size='large') 
            else:
                plt.text(x, y, label , rotation = 0, ha='center', va='center',size='small') 


In [30]:
def choose_data_three_per(fdf,fcol_list,x_value,item1, item2, item3):
    method = 'sum'
    
    data4chart1 = fdf[[fcol_list[0],fcol_list[1],fcol_list[2], 'count_of_appointments', x_value]]
    base = data4chart1.groupby([x_value])['count_of_appointments'].agg(method).to_frame()
       
    if item1 == 'All':
        if item2 == 'All':
            if item3 == 'All': # All are All
                data4chart1 = data4chart1.groupby([x_value])['count_of_appointments'].agg(method).to_frame() / base
            else: # NC is not All
                data4chart1 = data4chart1[(data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() / base
        else: # CT is not ALL
            if item3 == 'All': # CT is not ALL
                data4chart1 = data4chart1[(data4chart1[fcol_list[1]]==item2)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() / base
            else: # CT is not ALL & NC is not All
                data4chart1 = data4chart1[(data4chart1[fcol_list[1]]==item2) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() / base
    else: # SS are not All
        if item2 == 'All': # SS are not All
            if item3 == 'All': # SS are not All
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() / base
            else: # SS are not All & NC is not ALL
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() / base
        else: # SS are not All & & CT is not ALL
            if item3 == 'All': # SS in not ALL & CT is not ALL
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[1]]==item2)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() / base
            else: # SS in not ALL & CT is not ALL & NC is not All
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[1]]==item2) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() / base

    
    data4chart1.columns = ['count_of_appointments']

    return data4chart1

In [31]:
def choose_data_three_gen(fdf,fcol_list,x_value,item1, item2, item3,fincol='count_of_appointments',ftitle='out'):
    method = 'sum'
    coa = 'count_of_appointments'
     
    data4chart1 = fdf[[fcol_list[0],fcol_list[1],fcol_list[2], fincol,'count_of_appointments', x_value]]
        
    if item1 == 'All':
        if item2 == 'All':
            if item3 == 'All': # All are All
                base = data4chart1.groupby([x_value])['count_of_appointments'].agg(method).to_frame()
                data4chart1 = data4chart1.groupby([x_value])[fincol].agg(method).to_frame().div(base[coa], axis=0)
                # Info(data4chart1)
            else: # NC is not All
                base = data4chart1[(data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() 
                data4chart1 = data4chart1[(data4chart1[fcol_list[2]]==item3)].groupby([x_value])[fincol].agg(method).to_frame().div(base[coa], axis=0)
        else: # CT is not ALL
            if item3 == 'All': # CT is not ALL
                base = data4chart1[(data4chart1[fcol_list[1]]==item2)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() 
                data4chart1 = data4chart1[(data4chart1[fcol_list[1]]==item2)].groupby([x_value])[fincol].agg(method).to_frame().div(base[coa], axis=0)
            else: # CT is not ALL & NC is not All
                base = data4chart1[(data4chart1[fcol_list[1]]==item2) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame()
                data4chart1 = data4chart1[(data4chart1[fcol_list[1]]==item2) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])[fincol].agg(method).to_frame().div(base[coa], axis=0)
    else: # SS are not All
        if item2 == 'All': # SS are not All
            if item3 == 'All': # SS are not All
                base = data4chart1[(data4chart1[fcol_list[0]]==item1)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() 
                # Info(base)
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1)].groupby([x_value])[fincol].agg(method).to_frame().div(base[coa], axis=0)
            else: # SS are not All & NC is not ALL
                base = data4chart1[(data4chart1[col_list[0]]==item1) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() 
                data4chart1 = data4chart1[(data4chart1[col_list[0]]==item1) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])[fincol].agg(method).to_frame().div(base[coa], axis=0)
        else: # SS are not All & & CT is not ALL
            if item3 == 'All': # SS in not ALL & CT is not ALL
                base = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[1]]==item2)].groupby([x_value])['count_of_appointments'].agg(method).to_frame() 
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[1]]==item2)].groupby([x_value])[fincol].agg(method).to_frame().div(base[coa], axis=0)
            else: # SS in not ALL & CT is not ALL & NC is not All
                base = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[1]]==item2) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])[fincol].agg(method).to_frame() 
                data4chart1 = data4chart1[(data4chart1[fcol_list[0]]==item1) & (data4chart1[fcol_list[1]]==item2) & (data4chart1[fcol_list[2]]==item3)].groupby([x_value])['count_of_appointments'].agg(method).to_frame().div(base[coa], axis=0)

    # Info(data4chart1)
    # print(f'nfd')
    data4chart1.rename(columns={fincol:ftitle}, inplace=True)

    return data4chart1

In [32]:
def saferemove(fwlist,poplist):
    outlist = fwlist
    for i in poplist:
        if i in fwlist: outlist.remove(i)    
    return outlist   

In [33]:
def make_countplot_global(fdf,heading):
    # print(heading)
    # Info(fdf)
    chart1 = sns.countplot(x=heading,
                           data=fdf, 
                           order=fdf[heading].value_counts().index,
                           palette = 'coolwarm')

    chart1.tick_params(labelrotation=90)
    figsize=(6,16)
    sns.despine(left=True, bottom=False)  
    barplotlabels2(chart1,len(fdf[heading]))
    return chart1

## Work on Data

In [34]:
start_time = time.perf_counter()

df_ICBkey = districts[['SICBL22CD','SICBL22CDH','SICBL22NM','ICB22CD','ICB22CDH','ICB22NM','NHSER22CD','NHSER22CDH','NHSER22NM']].copy()
df_ICBkey.columns = ['sub_icb_location_ons_code','sub_icb_short_code','Sub_ICB_Location_name','icb_ons_code','short_code','ICB_Full_Name','region_ons_code','region_short_code','Region_Full_Name']
df_ICBkey = df_ICBkey.drop_duplicates().sort_values(by='icb_ons_code')


df_ICBkey['ICB_Name'] = df_ICBkey['ICB_Full_Name'].str.replace('Integrated Care Board','') 
df_ICBkey['ICB_Name'] = df_ICBkey['ICB_Name'].str.replace('NHS ','')
df_ICBkey['ICB_Name'] = df_ICBkey['ICB_Name'].str.replace('North','N.') 
df_ICBkey['ICB_Name'] = df_ICBkey['ICB_Name'].str.replace('South','S.') 
df_ICBkey['ICB_Name'] = df_ICBkey['ICB_Name'].str.replace('East','E.') 
df_ICBkey['ICB_Name'] = df_ICBkey['ICB_Name'].str.replace(' and ',' & ') 



print ('1', time.perf_counter() - start_time, "seconds")


# Info(df_ICBkey)

1 0.019039874998270534 seconds


In [35]:
# Making Dicts that help with mapping codes to names of places 
df_dict = df_ICBkey[['icb_ons_code','ICB_Name']].drop_duplicates().reset_index()

list1 = df_dict['icb_ons_code']
list2 = []
temp_dict = {}
for x in range(len(list1)):
    if list1[x] == 'E54000052': 
        list2.append('E54000063')
    elif list1[x] == 'E54000053': 
        list2.append('E54000064')
    else:
        list2.append(list1[x])
    temp_dict.update({list1[x]:list2[x]})

df_dict['icb_ons_code'] = df_dict['icb_ons_code'].map(temp_dict)
code2nam_dict_MM = df_dict.set_index("icb_ons_code")["ICB_Name"].to_dict()

df_dict = df_ICBkey[['icb_ons_code','Region_Full_Name']].copy()
df_dict['icb_ons_code'] = df_dict['icb_ons_code'].map(temp_dict)
code2nam_dict_MR = df_dict.set_index("icb_ons_code")["Region_Full_Name"].to_dict()


df_dict = df_ICBkey[['region_ons_code','Region_Full_Name']]
code2nam_dict_RR = df_dict.set_index("region_ons_code")["Region_Full_Name"].to_dict()


In [36]:
df_ar = pd.read_csv('appointments_regional.csv')

In [37]:
df_ad = pd.read_csv('actual_duration.csv')

In [38]:
df_nc = df_nc_holder.copy()

In [39]:
# Info(df_ar)
# Info(df_ad)
# Info(df_nc)

In [40]:
# display(df_arT['icb_ons_code'].drop_duplicates())

#### Formatting AR

In [41]:
# Inputing Clean Regions into AR

start_time = time.perf_counter()

cols_to_merge  = ['icb_ons_code','short_code', 'ICB_Full_Name','region_ons_code','region_short_code','Region_Full_Name','ICB_Name']
merge_df = df_ICBkey[cols_to_merge].drop_duplicates().copy()

# Info(merge_df)

df_arT = df_ar.merge(merge_df, on='icb_ons_code', how='inner')

cols = ['region_ons_code','region_short_code','Region_Full_Name', 
        'icb_ons_code', 'short_code', 'ICB_Full_Name', 'ICB_Name',
        'appointment_month', 'appointment_status', 'hcp_type', 'appointment_mode', 'time_between_book_and_appointment', 
        'count_of_appointments']
df_arT = df_arT.reindex(columns=cols)

# Info(df_arT)

df_ar = df_arT.copy()
print ('2', time.perf_counter() - start_time, "seconds")

2 0.6418431249985588 seconds


In [42]:
# Prep of Delay Data
ave_list = [1,18,4.5,25,11,35,0,'Nan']
max_list = [1,21,7,28,14,50,0,'Nan']
min_list = [1,15,2,22,8,28,0,'Nan']

metric=['time_between_book_and_appointment']
wlist=make_lists(df_ar,metric)
wlist[0].pop(0)

delay_dict = {wlist[0][i] : ave_list[i] for i in range(len(ave_list))}
df_ar['delay_anum'] = df_ar['time_between_book_and_appointment'].map(delay_dict) 
df_ar['delay_anum_float'] = pd.to_numeric(df_ar['delay_anum'], errors='coerce')
df_ar['delay_atot_float'] = df_ar['delay_anum_float'] * df_ar['count_of_appointments']
# print(df_ar.info())
# print(df_ar.describe())
print ('pre head', time.perf_counter() - start_time, "seconds")

# cols_to_drop =  ['region_short_code','short_code', 'ICB_Full_Name','delay_anum']
cols_to_drop =  ['delay_anum','delay_anum_float']

df_ar = df_ar.drop(cols_to_drop, axis=1)

# display(df_ar)

# Prep of Delay Data
print ('end', time.perf_counter() - start_time, "seconds")

pre head 0.9692344579962082 seconds
end 1.1645947079960024 seconds


In [43]:
df_ar['appointment_month'] = pd.to_datetime(df_ar['appointment_month'])

df_sar = df_ar.copy()

temp_dict = {'Face-to-Face':'Face-to-Face *', 'Home Visit':'Face-to-Face *', 
           'Telephone':'Telephone*', 'Unknown':'Unknown', 'Video/Online':'Telephone*'}

df_sar['appointment_mode*'] = df_sar.loc[:,'appointment_mode'].map(temp_dict)

df_ars = df_sar.copy()


df_sar = df_ar[df_sar['appointment_month']>='2021-08-01'].copy()
df_sar = df_sar[['icb_ons_code', 'ICB_Name', 'appointment_month', 'appointment_status', 'hcp_type', 'appointment_mode', 'count_of_appointments', 'delay_atot_float']].reset_index()

df_sar['appointment_month'] = df_sar['appointment_month'].astype(str).apply(lambda x: x[0:7])

# Info(df_sar)


 
df_sar.drop(['appointment_mode'],axis=1,inplace=True)

#### Formatting AD

In [44]:
# Inputing Clean Regions  AD
start_time = time.perf_counter()

merge_df = df_ICBkey.drop_duplicates().copy()

# Info(merge_df)

# Info(df_ad)



df_adT = df_ad.merge(merge_df, on='sub_icb_location_ons_code', how='inner')

df_adT['appointment_month'] = df_adT['appointment_date'].astype(str).apply(lambda x: x[-6:])


cols_to_drop = ['icb_ons_code_y','region_ons_code_y']
df_adT.drop(cols_to_drop, axis=1, inplace=True)
df_adT.rename(columns={'icb_ons_code_x':'icb_ons_code','region_ons_code_x':'region_ons_code'}, inplace=True)

# Info(df_adT)


cols = ['region_ons_code','region_short_code','Region_Full_Name', 
        'icb_ons_code', 'short_code', 'ICB_Full_Name', 'ICB_Name',
        'sub_icb_location_code','sub_icb_location_ons_code','Sub_ICB_Location_name',
        'appointment_date', 'actual_duration',
        'count_of_appointments']
df_adT = df_adT.reindex(columns=cols)

# Info(df_adT)

df_ad = df_adT.copy()

print ('2', time.perf_counter() - start_time, "seconds")

2 0.2669432090042392 seconds


In [45]:
# Prep of Duration Data
metric=['actual_duration']
wlist=make_lists(df_ad,metric)
wlist[0].pop(0)
# print(wlist)

ave_list = [45,25,8,'Nan',19,13,3]

delay_dict = {wlist[0][i] : ave_list[i] for i in range(len(ave_list))}
# print(delay_dict)
df_ad['dur_num'] = df_ad['actual_duration'].map(delay_dict) 
df_ad['dur_num_float'] = pd.to_numeric(df_ad['dur_num'], errors='coerce')
df_ad['dur_tot_float'] = df_ad['dur_num_float'] * df_ad['count_of_appointments']


# df_ar['delay_atot_float'] = df_ar['delay_anum_float'] * df_ar['count_of_appointments']

# Info(df_ad)


#### Formatting NC

In [46]:
# Inputing Clean Regions  NC
start_time = time.perf_counter()

df_ncT = df_nc.copy()



df_ncT['sub_icb_short_code'] = df_ncT.sub_icb_location_name.apply(lambda x: x[x.find('-')+2:])
df_ncT['sub_icb_short_code'] = df_ncT['sub_icb_short_code'].str.replace('n-Trent ICB - ','')


merge_df = df_ICBkey.drop_duplicates().copy()

# Info(merge_df)

df_ncT = df_ncT.merge(merge_df, on='sub_icb_short_code', how='inner')

cols_to_drop = ['icb_ons_code_y']
df_ncT.drop(cols_to_drop, axis=1, inplace=True)
df_ncT.rename(columns={'icb_ons_code_x':'icb_ons_code'}, inplace=True)

cols = ['region_ons_code','region_short_code','Region_Full_Name', 
        'icb_ons_code', 'short_code', 'ICB_Full_Name', 'ICB_Name',
        'sub_icb_location_ons_code','sub_icb_short_code','Sub_ICB_Location_name',
        'appointment_date', 'appointment_month','service_setting','context_type', 'national_category',
        'count_of_appointments']
df_ncT = df_ncT.reindex(columns=cols)

# Info(df_ncT)

df_nc = df_ncT.copy()

print ('2', time.perf_counter() - start_time, "seconds")

2 2.3705888750046142 seconds


In [47]:
start_time = time.perf_counter()



# Getting days of the week using the 'dayofweek' method, and then changing it to words
df_nc['weekday'] = df_nc['appointment_date'].dt.dayofweek
temp_dict = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
df_nc['weekday'] = df_nc['weekday'].map(temp_dict)
li = {'Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'}
df_nc['weekday'] = pd.Categorical(df_nc['weekday'], categories=li)



print (time.perf_counter() - start_time, "seconds")


0.21084829200117383 seconds


In [48]:
# Making Doctors file 

# Bring in the file
df_staffT = df_staff.copy()

# Bring in the right columns 
df_staffT = df_staffT[['SUB ICB code','Staff group','Full-time equivalent (FTE)']]
# Fix spelling mistakes
df_staffT['Staff group'] = df_staffT['Staff group'].str.replace('Admin/Non-clinical','Admin/Non-Clinical')
df_staffT.rename(columns={'SUB ICB code':'sub_icb_short_code'},inplace=True)
# Calculate the vales per region and pivot
df_staffT = df_staffT.groupby(['sub_icb_short_code','Staff group'])['Full-time equivalent (FTE)'].sum().reset_index()
df_staffT = df_staffT.pivot(values=["Full-time equivalent (FTE)"], index='sub_icb_short_code', columns="Staff group").reset_index()
df_staffT = df_staffT[df_staffT['sub_icb_short_code'] != 'Unknown']


cols =['sub_icb_short_code','FTE:Admin/Non-Clinical','FTE:Direct Patient Care','FTE:GP','FTE:Nurses']
df_staffT.columns=cols

merge_df = df_ICBkey.drop_duplicates().copy()


df_staffT = df_staffT.merge(merge_df, on='sub_icb_short_code', how='inner')

cols = ['region_ons_code','region_short_code','Region_Full_Name', 
        'icb_ons_code', 'short_code', 'ICB_Full_Name', 'ICB_Name',
        'sub_icb_location_ons_code','sub_icb_short_code','Sub_ICB_Location_name',
        'FTE:Admin/Non-Clinical','FTE:Direct Patient Care','FTE:GP','FTE:Nurses']
df_staffT = df_staffT.reindex(columns=cols)
df_staffT['FTE:Total'] = df_staffT[['FTE:Admin/Non-Clinical','FTE:Direct Patient Care','FTE:GP','FTE:Nurses']].sum(axis=1)
# Info(df_staffT)

In [49]:
# Making Patients file 
df_pat = patients.copy()

df_pat = df_pat.iloc[4:,:10]
 
df_pat.columns = [''] * len(df_pat.columns)

df_pat = df_pat.fillna('')
cols = list(df_pat.iloc[0,])
cols = ['sub_icb_short_code', 'name', 'All Patients', '0-4', '5-14', '15-44', '45-64', '65-74', '75-84', '85 and over']
# print(cols)

# df_pat.columns[0] = 'code'
df_pat.columns = cols
df_pat_tot = df_pat.loc[6,:].to_frame().T
# display(df_pat_tot.head())
df_pat = df_pat.iloc[5:,:10].reset_index()

merge_df = df_ICBkey.drop_duplicates().copy()
df_pat = df_pat.merge(merge_df, on='sub_icb_short_code', how='inner')

list1 = ['All Patients', '0-4', '5-14', '15-44', '45-64', '65-74', '75-84', '85 and over']
list2 = ['All Patients', 'Pat:0-4', 'Pat:5-14', 'Pat:15-44', 'Pat:45-64', 'Pat:65-74', 'Pat:75-84', 'Pat:85 and over']

temp_dict ={list1[i] : list2[i] for i in range(len(list1))}
df_pat.rename(columns=temp_dict,inplace=True)

df_pat[list2] = df_pat[list2].astype(int) 

cols = ['region_ons_code','region_short_code','Region_Full_Name', 
        'icb_ons_code', 'short_code', 'ICB_Full_Name', 'ICB_Name',
        'sub_icb_location_ons_code','sub_icb_short_code','Sub_ICB_Location_name',
        'All Patients', 'Pat:0-4', 'Pat:5-14', 'Pat:15-44', 'Pat:45-64', 'Pat:65-74', 'Pat:75-84', 'Pat:85 and over']
df_pat = df_pat.reindex(columns=cols)

list2 = ['Pat:65-74', 'Pat:75-84', 'Pat:85 and over']
df_pat['Pat:65 and over'] = df_pat[list2].sum(axis=1)

# THIS IS GOOD TO KEEP
# df_pat['Pat%:65 and over'] = df_pat['Pat%:65 and over'].apply(lambda x: '{:.3f}%'.format(x))

# temp_dict = {}
# for item in list2: temp_dict[item] = "${:,.2f}"
    
# # temp_dict = {list2[i]:'${:.3f}%' for i in range(len(list2))}
# # display(temp_dict)                                               
# df_pat.style.format(temp_dict)



#### Reduction of Items

In [50]:
# Preparing NC Data for pivot
print(df_nc['count_of_appointments'].sum())
list1 =list(df_nc.columns)
# print(list1)

# df_snc = df_nc[['appointment_date', 'service_setting', 'context_type', 'national_category','count_of_appointments', 'appointment_month','icb_ons_code', 'ICB_Name']].copy()
df_snc = df_nc.copy()

list1 = list(df_nc.loc[:,'service_setting'].drop_duplicates())
list2 = list(df_nc.loc[:,'context_type'].drop_duplicates())
list3 = list(df_nc.loc[:,'national_category'].drop_duplicates())

# print(list1)
outlist = ['Other','Other','General Practice','Unmapped','Other']
temp_dict ={list1[i] : outlist[i] for i in range(len(outlist))}
df_snc['service_setting*'] = df_snc['service_setting'].map(temp_dict)
df_snc = df_snc.drop(['service_setting'],axis=1)

# print(list2)
outlist = ['Care Related Encounter', 'Bad mapping*', 'Bad mapping*']
temp_dict ={list2[i] : outlist[i] for i in range(len(outlist))}
df_snc['context_type*'] = df_snc['context_type'].map(temp_dict) 
df_snc = df_snc.drop(['context_type'],axis=1)

print(temp_dict)

# print(list3)
outlist = ['Other*', 'Planned Clinics*', 'Other*', 
           'General Consultation Acute', 'Other*', 'Other*', 
           'Bad mapping*', 'Clinical Triage', 'Planned Clinics*', 'Bad mapping*', 
           'Other*', 
           'General Consultation Routine', 
           'Other*', 
           'Other*', 'Other*', 
           'Other*', 'Other*', 'Other*']

temp_dict ={list3[i] : outlist[i] for i in range(len(outlist))}
df_snc['national_category*'] = df_snc['national_category'].map(temp_dict) 
df_snc = df_snc.drop(['national_category'],axis=1)

# print(temp_dict)

list1 =list(df_snc.columns)
# print(list1)

list1 = ['appointment_month', 'icb_ons_code', 'ICB_Name', 'service_setting*', 'context_type*', 'national_category*']

df_snc['appointment_month'] = df_snc['appointment_month'].astype(str)

df_ncs = df_snc.copy()

df_snc = df_snc.groupby(list1).agg({'count_of_appointments':'sum'}).reset_index()

# Info(df_snc)

296046770
{'Care Related Encounter': 'Care Related Encounter', 'Unmapped': 'Bad mapping*', 'Inconsistent Mapping': 'Bad mapping*'}


#### Pivot work

In [51]:
# Preparing AR Data for pivot

df_ar['appointment_month'] = pd.to_datetime(df_ar['appointment_month'])
df_sar = df_ar[df_ar['appointment_month']>='2021-08-01'].copy()
df_sar = df_sar[['icb_ons_code', 'ICB_Name', 'appointment_month', 'appointment_status', 'hcp_type', 'appointment_mode', 'count_of_appointments', 'delay_atot_float']].reset_index()

df_sar['appointment_month'] = df_sar['appointment_month'].astype(str).apply(lambda x: x[0:7])

# Info(df_sar)

# tempicol = ['icb_ons_code','appointment_month']
# df_sar = df_sar.merge(ar_mloc, on=tempicol, how='left')


list1 = list(df_ar.loc[:,'appointment_mode'].drop_duplicates())

temp_dict = {'Face-to-Face':'Face-to-Face *', 'Home Visit':'Face-to-Face *', 
           'Telephone':'Telephone*', 'Unknown':'Unknown', 'Video/Online':'Telephone*'}

df_sar['appointment_mode*'] = df_sar.loc[:,'appointment_mode'].map(temp_dict) 
df_sar.drop(['appointment_mode'],axis=1,inplace=True)
df_sar.drop(columns='index',inplace=True)

# Info(df_sar.head())



In [52]:
# Pivoting 

start_time = time.perf_counter()

def make_big_pivot(fdf,fcolumns, findexes, fvalues):
    for i in range(len(tempcol)): 
        # print(i)
        df_out = fdf.pivot_table(index=findexes, columns=fcolumns[i], values=fvalues, aggfunc='sum')
        # print(df_out.loc[:,'count_of_appointments'].sum().sum())
        df_out = df_out.rename(mapper={str(fvalues[0]): str(fcolumns[i])}, axis=1)
        if i == 0:
            big_out = df_out.copy()
        else:
            big_out = big_out.join([df_out])
    return big_out


tempcol = ['service_setting*', 'context_type*', 'national_category*']
tempicol = ['icb_ons_code', 'appointment_month']
tempval = ['count_of_appointments']
df_snc_mod = make_big_pivot(df_snc,tempcol,tempicol, tempval).reset_index()
df_snc_mod = df_snc_mod.set_index([df_snc_mod.columns[0],df_snc_mod.columns[1]])
# Info(df_snc_mod)  
# df_snc_mod.index.name 


# tempicol = ['icb_ons_code', 'ICB_Name', 'appointment_month'] + ['mon_count_of_appointments',
#        'mon_delay_atot_float', 'mon_ave_delay']

tempicol = ['icb_ons_code', 'ICB_Name', 'appointment_month'] 
tempcol = ['appointment_status', 'hcp_type', 'appointment_mode*']
tempval = ['count_of_appointments']
df_sar_mod = make_big_pivot(df_sar,tempcol,tempicol, tempval).reset_index()
df_sar_mod = df_sar_mod.set_index([df_sar_mod.columns[0],df_sar_mod.columns[2]])
# df_sar_mod.index.name
# Info(df_sar_mod)  
    
df_arnc_modo1 = df_sar_mod.join([df_snc_mod])
# Info(df_arnc_modo1)

print (time.perf_counter() - start_time, "seconds")

    

0.39412854200054426 seconds


In [53]:
# Prepare and add the Delay data
start_time = time.perf_counter()
heading='icb_ons_code'
huein='appointment_month'

# 'delay_atot_float'

ar_mloc = get_twoway_data_avdel(df_ar,heading,huein)
ar_mloc = ar_mloc.set_index(['icb_ons_code', 'appointment_month']).reset_index()
ar_mloc = ar_mloc[ar_mloc['appointment_month']>='2021-08-01']
ar_mloc['appointment_month'] = ar_mloc['appointment_month'].astype(str).apply(lambda x: x[0:7])

temp_dict = {'icb_ons_code':'icb_ons_code', 'appointment_month':'appointment_month',
             'count_of_appointments':'mon_count_of_appointments',
             'delay_atot_float':'mon_delay_atot_float', 'ave_delay':'mon_ave_delay'}
ar_mloc.rename(columns=temp_dict,inplace=True)
ar_mloc = ar_mloc.set_index([ar_mloc.columns[0],ar_mloc.columns[1]])

header = pd.MultiIndex.from_product([['monthly_values'],
                                     ['mon_count_of_appointments', 'mon_delay_atot_float', 'mon_ave_delay']])
ar_mloc.columns=header
ar_mloc = ar_mloc.reset_index()
ar_mloc = ar_mloc.set_index([ar_mloc.columns[0],ar_mloc.columns[1]])
# Info(ar_mloc)


# Info(df_arnc_mod)
# print(list([ar_mloc.columns]))

print (time.perf_counter() - start_time, "seconds")



0.1711584579970804 seconds


In [54]:
def sort_data_4chart2(fdata4chart1,fheading,fincol,ftype):
    
    print(fheading,fincol,ftype)
    tempdata = fdata4chart1.groupby(fheading).agg(ftype).copy()[fincol].to_frame().reset_index()    

    if fheading == 'time_between_book_and_appointment':
        li = ['Same Day','1 Day', '2 to 7 Days', '8  to 14 Days',
              '15  to 21 Days', '22  to 28 Days', 'More than 28 Days', 
    
              'Unknown / Data Quality']
    else:
        li = list(tempdata.sort_values(by=tempdata.columns[1], ascending=False)[fheading])   
    return li

In [55]:
# Info(df_sar_mod)
# Info(df_snc_mod)
# Info(ar_mloc)

In [56]:
# Joining AR and NC
df_step1 = df_sar_mod.join([df_snc_mod])
# Joining AR and Delay
df_arnc_mod = df_step1.join([ar_mloc])
# del df_step1



In [57]:
# Add Regional Data 

df_arnc_mod = df_arnc_mod.reset_index()
list1 = ["icb_ons_code","Region_Full_Name"]
df_dict = df_ICBkey[list1].drop_duplicates()
result_dict = df_dict.set_index("icb_ons_code")["Region_Full_Name"].to_dict()
df_arnc_mod['Region'] = df_arnc_mod[df_arnc_mod.columns[0]].map(result_dict) 


# Info(df_arnc_mod)


In [58]:
# Geo Cleaning
df_regions.rename(columns={'NHSER22CD': 'region_ons_code'}, inplace=True)
df_ICBs.rename(columns={'ICB23CD': 'icb_ons_code'}, inplace=True)
df_SubICBs.rename(columns={'SICBL23CD': 'sub_icb_location_ons_code'}, inplace=True)

In [59]:
# Main Function that Gets the Data for Maps and Main Scatter Plot  
def get_map_df(fregvar,fdf_ar,fdf_ad,fdf_nc):

    start_time = time.perf_counter()

    pd.options.display.float_format = '{:,.4f}'.format
#     pd.options.display.int64_format = '{:,.0f}'.format
    
    listpat1 = ['All Patients','Pat:0-4', 'Pat:5-14', 'Pat:15-44', 'Pat:45-64', 'Pat:65-74', 'Pat:75-84', 'Pat:85 and over','Pat:65 and over']
#     listpat2 = ['Pat%:0-4', 'Pat%:5-14', 'Pat%:15-44%', 'Pat%:45-64', 'Pat%:65-74', 'Pat%:75-84', 'Pat%:85 and over','Pat%:65 and over']
    listpat2 = []
    listpatT = fregvar + listpat1 + listpat2 + ['region_short_code']

    
    df_for_map_pat = df_pat[listpatT]
    if fregvar != ['sub_icb_location_ons_code']:
        df_for_map_pat = df_for_map_pat.groupby(fregvar)[listpat1 + listpat2].sum()
    
    liststaff1 = ['FTE:Admin/Non-Clinical','FTE:Direct Patient Care','FTE:GP','FTE:Nurses','FTE:Total']
    liststaff2 = ['FTE%:Admin/Non-Clinical','FTE%:Direct Patient Care','FTE%:GP','FTE%:Nurses','FTE%:Total']

    liststaffT = fregvar + liststaff1 + ['region_short_code']

    
#     print(liststaffT)
    
    df_for_map_staff = df_staffT[liststaffT]
    if fregvar != ['sub_icb_location_ons_code']:
        df_for_map_staff = df_for_map_staff.groupby(fregvar)[liststaff1].sum()
    
# Merge 1: Patients & Staff
    df_for_map1T = df_for_map_pat.merge(df_for_map_staff, on=fregvar, how='left')

# Additional work on Patients     
    listpat1 = ['Pat:0-4', 'Pat:5-14', 'Pat:15-44', 'Pat:45-64', 'Pat:65-74', 'Pat:75-84', 'Pat:85 and over','Pat:65 and over']
    listpat2 = ['Pat%:0-4', 'Pat%:5-14', 'Pat%:15-44', 'Pat%:45-64', 'Pat%:65-74', 'Pat%:75-84', 'Pat%:85 and over','Pat%:65 and over']

    
    df_for_map1T[listpat2] = df_for_map1T[listpat1].div(df_for_map1T['All Patients'], axis=0)
    
    list1 = ['Pat:0-4', 'Pat:5-14']
    df_for_map1T['Pat:15 and below'] = df_for_map1T[list1].sum(axis=1)
    df_for_map1T['Pat%:15 and below'] =  df_for_map1T['All Patients']
    
# Additional work on FTEs     
    df_for_map1T[liststaff2] = df_for_map1T[liststaff1].div(df_for_map1T['FTE:Total'], axis=0)
    
        
    
# --------------------------Work on AR---------------------
    heading1 = []
    
    arlist1 = ['region_ons_code', 'region_short_code', 'Region_Full_Name', 'icb_ons_code', 'short_code', 'ICB_Full_Name', 
               'ICB_Name', 'appointment_month', 'appointment_status', 'hcp_type', 'appointment_mode', 'time_between_book_and_appointment'] 
    arlist2 = fregvar + heading1 
    arlist3 = ['count_of_appointments', 'delay_atot_float']
    
    if fregvar != ['sub_icb_location_ons_code']:
        # df_ar_map = get_twoway_df
        df_ar_map =fdf_ar.groupby(arlist2)[arlist3].agg('sum').reset_index()
        df_ar_map['count_of_appointments_ave_ar'] = df_ar_map['count_of_appointments'] / 30
        df_ar_map['average_delay'] = df_ar_map['delay_atot_float']/df_ar_map['count_of_appointments']
        df_ar_map.drop(columns=arlist3, inplace=True) 
        df_for_map1T = df_ar_map.merge(df_for_map1T, on=fregvar, how='left')


# Merge 2

# --------------------------Work on AD---------------------

    heading2 = []


    adlist1 = ['region_ons_code', 'region_short_code', 'Region_Full_Name', 'icb_ons_code', 'short_code', 
               'ICB_Full_Name', 'ICB_Name', 'sub_icb_location_code', 'sub_icb_location_ons_code', 'Sub_ICB_Location_name', 
               'appointment_date', 'actual_duration']
    adlist4 = ['dur_num', 'dur_num_float', 'dur_tot_float']
    adlist2 = fregvar + heading2
    adlist3 = ['count_of_appointments'] + ['dur_tot_float']
#     + adlist4
    
    
    df_ad_map =fdf_ad.groupby(arlist2)[adlist3].agg('sum').reset_index()
    df_ad_map['count_of_appointments_ave_ad'] = df_ad_map['count_of_appointments'] / 7
    df_ad_map['average_duration'] = df_ad_map['dur_tot_float']/df_ad_map['count_of_appointments']

#     Info(df_ad_map)
    
    df_ad_map.drop(columns=adlist3,inplace=True) 

# Merge 3
    df_for_map1T = df_ad_map.merge(df_for_map1T, on=fregvar, how='left')

    
# --------------------------Work on NC---------------------

    heading3 = []


    nclist2 = fregvar + heading3
    nclist3 = ['count_of_appointments']
    # print(fregvar) 

    # Info(fdf_nc)
    df_nc_map = fdf_nc.groupby(nclist2)[nclist3].agg('sum').reset_index()
    df_nc_map['count_of_appointments_ave_nc'] = df_nc_map['count_of_appointments'] / 11
    df_nc_map.drop(columns=nclist3) 
 
# Merge 4
    df_for_map1T = df_nc_map.merge(df_for_map1T, on=fregvar, how='left')
    
  
# Additonal FTE:Staff workings 

#     Info(df_for_map1T)

    oldlist = ['All Patients','Pat:15-44', 'Pat:45-64', 'Pat:65 and over','Pat:15 and below'] 
    if fregvar != ['sub_icb_location_ons_code']:
        oldlist = oldlist + ['count_of_appointments_ave_ar'] + ['count_of_appointments_ave_ad'] + ['count_of_appointments_ave_nc']
    else:  
        oldlist = oldlist  + ['count_of_appointments_ave_ad'] + ['count_of_appointments_ave_nc']
     
    newlist = ['Patients per FTE','Pat:15-44 per FTE', 'Pat:45-64 per FTE', 'Pat:65 and over per FTE','Pat:15 and below per FTE']

    if fregvar != ['sub_icb_location_ons_code']:    
        newlist = newlist + ['Ave_apt_recent (AR) per FTE'] + ['Ave_apt_attened (AD) per FTE'] + ['Ave_apt_long (NC) per FTE']
    else: 
        newlist = newlist + ['Ave_apt_attened (AD) per FTE'] + ['Ave_apt_long (NC) per FTE']
         
    df_for_map1T[newlist] = df_for_map1T[oldlist].div(df_for_map1T['FTE:Total'], axis=0)
    
    newlist = ['Patients per GP','Pat:15-44 per GP', 'Pat:45-64 per GP', 'Pat:65 and over per GP','Pat:15 and below per GP']
    if fregvar != ['sub_icb_location_ons_code']:
        newlist = newlist + ['Ave_apt_recent (AR) per GP'] + ['Ave_apt_attened (AD) per GP'] + ['Ave_apt_long (NC) per GP']
    else:  
        newlist = newlist + ['Ave_apt_attened (AD) per GP'] + ['Ave_apt_long (NC) per GP']
    
    
    df_for_map1T[newlist] = df_for_map1T[oldlist].div(df_for_map1T['FTE:GP'], axis=0)
    

    newlist = ['Patients per GP','Pat:15-44 per Admin', 'Pat:45-64 per Admin', 'Pat:65 and over per Admin','Pat:15 and below per Admin']
    if fregvar != ['sub_icb_location_ons_code']:
        newlist = newlist + ['Ave_apt_recent (AR) per Admin'] + ['Ave_apt_attened (AD) per Admin'] + ['Ave_apt_long (NC) per Admin']
    else:  
        newlist = newlist  + ['Ave_apt_attened (AD) per Admin'] + ['Ave_apt_long (NC) per Admin']

    df_for_map1T[newlist] = df_for_map1T[oldlist].div(df_for_map1T['FTE%:Admin/Non-Clinical'], axis=0)

    
# Additional work on Patients (redux)    
    listpat1 = ['count_of_appointments_ave_ad'] + ['count_of_appointments_ave_nc']
    listpat2 = ['count_of_appointments_ave_ad per Pats'] + ['count_of_appointments_ave_nc per Pats']

    # display(df_for_map1T[count_of_appointments_ave_ar])
    # Info(df_for_map1T)
    df_for_map1T[listpat2] = df_for_map1T[listpat1].div(df_for_map1T['All Patients'], axis=0)

    if fregvar != ['sub_icb_location_ons_code']:
        listpat1 = ['count_of_appointments_ave_ar']
        listpat2 = ['count_of_appointments_ave_ar per Pats'] 
        df_for_map1T[listpat2] = df_for_map1T[listpat1].div(df_for_map1T['All Patients'], axis=0)

    # count_of_appointments_ave_nc
    
    # df_full_cols = df_for_map1T
#     Info(df_full_cols)
 

    dp1 = ['Pat:0-4', 'Pat:5-14', 'Pat:15-44', 'Pat:45-64', 'Pat:65-74', 'Pat:75-84', 
           'Pat:85 and over', 'Pat:65 and over','Pat:15 and below']
    dp2 = ['Pat%:0-4', 'Pat%:5-14','Pat%:65-74', 'Pat%:75-84', 'Pat%:85 and over']
    dp3 = ['FTE%:Total']
    dp4 = ['FTE:Admin/Non-Clinical', 'FTE:Direct Patient Care', 'FTE:GP', 'FTE:Nurses']
    droplist = dp1 + dp2 + dp3 + dp4
    
    df_for_map1T.drop(columns=droplist,inplace=True) 
    
    df_dict = df_ICBkey[['icb_ons_code','ICB_Name']]
    code2nam_dict_MM = df_dict.set_index("icb_ons_code")["ICB_Name"].to_dict()
    
    #Clean up for Maps codes for maps    
    if fregvar == ['icb_ons_code']:
        list1 = df_for_map1T['icb_ons_code'].drop_duplicates()
        # print(list1)
        list2 = []
        temp_dict = {}
        for x in range(len(list1)):
            # print(x, list1[x])
            if list1[x] == 'E54000052': 
                list2.append('E54000063')
            elif list1[x] == 'E54000053': 
                list2.append('E54000064')
            else:
                list2.append(list1[x])
            temp_dict.update({list1[x]:list2[x]})
        
        # print(temp_dict)
        df_for_map1T['icb_ons_code'] = df_for_map1T['icb_ons_code'].map(temp_dict)
    
    #       df_for_map1T.rename(columns = {'icb_ons_code':'icb_ons_code*'})
    last_order = fregvar + ['average_delay', 'average_duration', 'count_of_appointments','count_of_appointments_ave_nc', 
                  'count_of_appointments_ave_ad','count_of_appointments_ave_ar','count_of_appointments_ave_ad per Pats',
                  'count_of_appointments_ave_nc per Pats','count_of_appointments_ave_ar per Pats', 'Ave_apt_long (NC) per FTE', 
                  'Ave_apt_recent (AR) per FTE', 'Ave_apt_attened (AD) per FTE','Ave_apt_long (NC) per GP', 'Ave_apt_recent (AR) per GP',
                  'Ave_apt_attened (AD) per GP', 'Ave_apt_long (NC) per Admin','Ave_apt_recent (AR) per Admin','Ave_apt_attened (AD) per Admin', 
                  'All Patients', 'Pat%:15 and below','Pat%:15-44', 'Pat%:45-64','Pat%:65 and over', 'Patients per FTE', 'Pat:15 and below per FTE', 
                  'Pat:15-44 per FTE', 'Pat:45-64 per FTE','Pat:65 and over per FTE', 'Patients per GP', 'Pat:15 and below per GP','Pat:15-44 per GP', 
                  'Pat:45-64 per GP', 'Pat:65 and over per GP', 'Pat:15 and below per Admin', 'Pat:15-44 per Admin', 'Pat:45-64 per Admin', 
                  'Pat:65 and over per Admin','FTE:Total', 'FTE%:Admin/Non-Clinical', 'FTE%:Direct Patient Care', 'FTE%:GP', 'FTE%:Nurses']
    
    if fregvar == ['sub_icb_location_ons_code']:
        poplist = ['average_delay', 'count_of_appointments_ave_ar', 'count_of_appointments_ave_ar per Pats', 'Ave_apt_recent (AR) per FTE', 'Ave_apt_recent (AR) per GP', 'Ave_apt_recent (AR) per Admin']
        last_order = saferemove(last_order,poplist)
    
    df_for_map1T = df_for_map1T[last_order]    
            
#     Info(df_for_map1T)
    return  df_for_map1T


df_mapR = get_map_df(['region_ons_code'],df_ar,df_ad,df_nc)
df_mapM = get_map_df(['icb_ons_code'],df_ar,df_ad,df_nc)
df_mapS = get_map_df(['sub_icb_location_ons_code'],df_ar,df_ad,df_nc)

# def get_twoway_df(fdf,fheading, fhuein=[], fexclude1 = [], fexclude2 = []):



In [60]:
# print(df_ar['count_of_appointments'].sum())
# print(df_ars['count_of_appointments'].sum())

# print(df_nc['count_of_appointments'].sum())
# print(df_ncs['count_of_appointments'].sum())

# Info(df_ncs)

In [61]:
# Getting rid of bad data


# print(df_ar['count_of_appointments'].sum())
# print(df_ars['count_of_appointments'].sum())
df_nmapar = df_ars[df_ars['hcp_type']!='Unknown']
# print(df_nmapar['count_of_appointments'].sum())
df_nmapar = df_nmapar[df_nmapar['appointment_mode*']!='Unknown']
# print(df_nmapar['count_of_appointments'].sum())

# print(df_nc['count_of_appointments'].sum())
# print(df_ncs['count_of_appointments'].sum())
df_nmapnc = df_ncs[df_ncs['service_setting*']!='Unmapped']
# print(df_nmapnc['count_of_appointments'].sum())
df_nmapnc = df_nmapnc[df_ncs['context_type*']!='Bad mapping*']
# print(df_nmapnc['count_of_appointments'].sum())

df_mapRs = get_map_df(['region_ons_code'],df_nmapar,df_ad,df_nmapnc)
df_mapMs = get_map_df(['icb_ons_code'],df_nmapar,df_ad,df_nmapnc)
df_mapSs = get_map_df(['sub_icb_location_ons_code'],df_nmapar,df_ad,df_nmapnc)




## Code for Charts

#### Main Map Code 

In [62]:
# Main Map Code

regseglist = ['All'] + (list(df_staffT['Region_Full_Name'].drop_duplicates()))

def make_map_child(fcolumn,fdatain,fregvar):    

    if fregvar[0] == 'sub_icb_location_ons_code':
        map_data = fdatain.merge(df_mapS[[fregvar[0],name2col(fcolumn)]],on=fregvar[0],how='left')
    else:
         map_data = fdatain.merge(df_mapM[[fregvar[0],name2col(fcolumn)]],on=fregvar[0],how='left')
    
    colpal = sns.color_palette("coolwarm", as_cmap=True)    
    map1 = map_data.plot(column=name2col(fcolumn),figsize=(20,12),legend=True,cmap=colpal)
    format_chart1(map1)
    map1.axes.get_xaxis().set_visible(False)
    map1.axes.get_yaxis().set_visible(False)

    if len(fdatain[fregvar[0]]) < 15:
        titlestr = f'{fcolumn} \n (Detail ICB Data)'
    else:
        titlestr = f'{fcolumn} \n (Color based on ICBs, Data based on Regions)'    
    plt.title(titlestr, fontsize=20)
    
    return map1

def MapTest():
    wlist = list(df_mapM.columns)
    to_drop_list = ['icb_ons_code']
    wlist = saferemove(wlist, to_drop_list)
    for i in range(len(wlist)):
        wlist[i] = col2name(wlist[i])

    
    def make_interaction_map(item1,regmap,regseg):
        start_time2 = time.perf_counter()
        if regmap == "R":
            wlist = list(df_mapR.columns)
            dfgeo = df_regions.copy()
            regvar = ['region_ons_code']
            wlist.remove('region_ons_code')
        elif regmap == "M":
            wlist = list(df_mapM.columns)
            dfgeo = df_ICBs.copy()
            regvar = ['icb_ons_code']
            wlist.remove('icb_ons_code')

            if regseg != 'All':
                dfgeo['Region_Name'] = dfgeo['icb_ons_code'].map(code2nam_dict_MR) 
                dfgeo = dfgeo[dfgeo['Region_Name'] == regseg]
        
        elif regmap == "S":
            wlist = list(df_mapS.columns)
            dfgeo = df_SubICBs.copy()
            regvar = ['sub_icb_location_ons_code']        
            wlist.remove('sub_icb_location_ons_code')

        plt.ioff()
        make_map_child(item1,dfgeo,regvar)
        if regseg == 'All': 
            maplabels1('R',item1)
        else:
            maplabels1(regmap,item1,dfgeo)
        plt.ion()
        plt.show()
           
    plt.ion()

    out1 = widgets.Output()
    out2 = widgets.Output()
    out3 = widgets.Output()
    out4 = widgets.Output()
    out5 = widgets.Output()
    out6 = widgets.Output()
    out7 = widgets.Output()
    out8 = widgets.Output()
    out9 = widgets.Output()

    tab = widgets.Tab(children = [out1, out2, out3,out4, out5, out6, out7, out8, out9])
    tab.set_title(0,'England')
    tab.set_title(1,'N. East & Yorkshire')
    tab.set_title(2,'North West')
    tab.set_title(3,'Midlands')
    tab.set_title(4,'East of England') 
    tab.set_title(5,'South East') 
    tab.set_title(6,'South West') 
    tab.set_title(7,'London')
    tab.set_title(8,'Sub-ICB Analysis')

        
    display(tab)

    box_layout1 = widgets.Layout(justify_content='center')

    app_layout = widgets.Layout(
                # display='flex',
                justify_content='center',
                # flex_flow='row nowrap',
                # align_items='center',
                # border='none',
                # width='100%',
                margin='5px 5px 5px 5px')
    
    style = {'description_width': 'initial'}
    
    with out1:   
        drop_downMain = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_downMain.children[0].value = col2name('count_of_appointments')
        drop_downMain.children[0].description = 'Choose Dataseries:'
        drop_downMain.children[0].style = style
        drop_downMain.children[1].value = 'M'
        drop_downMain.children[2].value = regseglist[0]
        
        app1 = widgets.HBox([drop_downMain.children[0]],layout = box_layout1)
        app2 = widgets.VBox([app1, drop_downMain.children[-1]], layout = box_layout1)

        display(app2)

    with out2:   
        drop_down1 = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_down1.children[0].value = col2name('count_of_appointments')
        drop_down1.children[0].description = 'Choose Dataseries:'
        drop_down1.children[0].style = style
        drop_down1.children[1].value = 'M'
        drop_down1.children[2].value = regseglist[1]
        
        app11 = widgets.HBox([drop_down1.children[0]],layout = box_layout1)
        app21 = widgets.VBox([app11, drop_down1.children[-1]], layout = box_layout1)

        display(app21)

    with out3:   
        drop_down1 = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_down1.children[0].value = col2name('count_of_appointments')
        drop_down1.children[0].description = 'Choose Dataseries:'
        drop_down1.children[0].style = style
        drop_down1.children[1].value = 'M'
        drop_down1.children[2].value = regseglist[2]
        
        app11 = widgets.HBox([drop_down1.children[0]],layout = box_layout1)
        app21 = widgets.VBox([app11, drop_down1.children[-1]], layout = box_layout1)

        display(app21)


    with out4:   
        drop_down1 = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_down1.children[0].value = col2name('count_of_appointments')
        drop_down1.children[0].description = 'Choose Dataseries:'
        drop_down1.children[0].style = style
        drop_down1.children[1].value = 'M'
        drop_down1.children[2].value = regseglist[3]
        
        app11 = widgets.HBox([drop_down1.children[0]],layout = box_layout1)
        app21 = widgets.VBox([app11, drop_down1.children[-1]], layout = box_layout1)

        display(app21)

    with out5:   
        drop_down1 = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_down1.children[0].value = col2name('count_of_appointments')
        drop_down1.children[0].description = 'Choose Dataseries:'
        drop_down1.children[0].style = style
        drop_down1.children[1].value = 'M'
        drop_down1.children[2].value = regseglist[4]
        
        app11 = widgets.HBox([drop_down1.children[0]],layout = box_layout1)
        app21 = widgets.VBox([app11, drop_down1.children[-1]], layout = box_layout1)

        display(app21)


    with out6:   
        drop_down1 = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_down1.children[0].value = col2name('count_of_appointments')
        drop_down1.children[0].description = 'Choose Dataseries:'
        drop_down1.children[0].style = style
        drop_down1.children[1].value = 'M'
        drop_down1.children[2].value = regseglist[5]
        
        app11 = widgets.HBox([drop_down1.children[0]],layout = box_layout1)
        app21 = widgets.VBox([app11, drop_down1.children[-1]], layout = box_layout1)

        display(app21)

    with out7:   
        drop_down1 = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_down1.children[0].value = col2name('count_of_appointments')
        drop_down1.children[0].description = 'Choose Dataseries:'
        drop_down1.children[0].style = style
        drop_down1.children[1].value = 'M'
        drop_down1.children[2].value = regseglist[6]
        
        app11 = widgets.HBox([drop_down1.children[0]],layout = box_layout1)
        app21 = widgets.VBox([app11, drop_down1.children[-1]], layout = box_layout1)

        display(app21)

    with out8:   
        drop_down1 = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_down1.children[0].value = col2name('count_of_appointments')
        drop_down1.children[0].description = 'Choose Dataseries:'
        drop_down1.children[0].style = style
        drop_down1.children[1].value = 'M'
        drop_down1.children[2].value = regseglist[7]
        
        app11 = widgets.HBox([drop_down1.children[0]],layout = box_layout1)
        app21 = widgets.VBox([app11, drop_down1.children[-1]], layout = box_layout1)

        display(app21)

    with out9:   
        drop_down1 = widgets.interactive(make_interaction_map, item1=wlist,regmap=['R','M','S'],regseg = regseglist)
        drop_down1.children[0].value = col2name('count_of_appointments')
        drop_down1.children[0].description = 'Choose Dataseries:'
        drop_down1.children[0].style = style
        drop_down1.children[1].value = 'S'
        drop_down1.children[2].value = regseglist[0]
        
        app11 = widgets.HBox([drop_down1.children[0]],layout = box_layout1)
        app21 = widgets.VBox([app11, drop_down1.children[-1]], layout = box_layout1)

        
        display(app21)


    
    
 



#### Time Plots for AR

In [63]:
def make_three_dropdown_bar_time_ser_delay(fdf,fcol_list,x_value,per_sw=0):
    wlist = make_lists(fdf,fcol_list)


    def make_interaction(item1, item2, item3):
        if per_sw == 0:
            datain = choose_data_three_gen(fdf,fcol_list,x_value,item1, item2, item3, fincol='delay_atot_float')
        elif per_sw == 1:
            print('Your are Boob')
        
        if datain.iloc[:,0].sum() != 0:
    #         display(list(datain.iloc[:,0])) # Don't delete yet
            chart1 = make_barplot(datain)
            titlestr = f"Average Duration between Booking and Appointment \n with Appointment Status: {col2name(item1)}, HCP Type: {col2name(item2)} & 'Appointment Mode: {col2name(item3)}"
            plt.title(titlestr, fontsize=20)
            # chart1.set(title= titlestr)
            plt.show()
        else:
            display('No Data in this combination')


    
    def make_barplot(data1):
        chart1 = sns.barplot(x=data1.index, 
                             y=data1.columns[0], 
                             data=data1,
                             palette='light:b')
        sns.despine(left=True, bottom=False)
        chart1.tick_params(labelrotation=90)
        chart1.set(ylabel= 'Average Delay of Appointment')
        chart1.set(xlabel= 'Month')
        ax = plt.gca()
        ax.set_yticklabels([])       
        
        if per_sw == 0:
            barplotlabels(chart1)
        elif per_sw == 1:
            barplotlabelsper(chart1)
        return chart1    

    drop_down = widgets.interactive(make_interaction, item1=wlist[0], item2=wlist[1], item3=wlist[2])

    la1 = widgets.Layout(border='solid 2px')

    style = {'description_width': 'initial'}
    drop_down.children[0].description = fcol_list[0].replace('_',' ').title()
    drop_down.children[0].style = style
    drop_down.children[0].value = 'All'

    drop_down.children[1].description = fcol_list[1].replace('_',' ').title()
    drop_down.children[1].style = style
    drop_down.children[1].value = 'All'


    drop_down.children[2].description = fcol_list[2].replace('_',' ').title()
    drop_down.children[2].style = style
    drop_down.children[2].value = 'All'


    app1 = widgets.HBox([drop_down.children[0],drop_down.children[1],drop_down.children[2]])
    app2 = widgets.VBox([app1,drop_down.children[-1]], layout=la1)

    display(app2)



# make_three_dropdown_bar_time_ser(df,col_list,x_value)



In [64]:
def DurationPlay(): 
    sns.color_palette("light:b", as_cmap=True)
    sns.set(rc={"figure.figsize":(16, 5)}) #width=3, #height=4
    sns.set_style('white')
    
    #Bring in the right DataFrame 
    df = df_ars.copy()
    
    # Format the month variable so it is string and shorter
    x_value = 'appointment_month'
    df[x_value] = df [x_value].astype(str).apply(lambda x: x[0:7])
    
    # Bring in the right columns to investigate
    col_list = ['appointment_status','hcp_type','appointment_mode','time_between_book_and_appointment']
    col_list = ['appointment_status','hcp_type','appointment_mode']
    
    # test_df = choose_data_three_gen(df,col_list,x_value,item1, item2, item3, fincol='delay_atot_float')
    # Info(test_df)
    
    make_three_dropdown_bar_time_ser_delay(df,col_list,x_value)



In [65]:
def make_three_dropdown_bar_time_ser(fdf,fcol_list,x_value,per_sw=0):
    wlist = make_lists(fdf,fcol_list)


    def make_interaction(item1, item2, item3):
        if per_sw == 0:
            datain = choose_data_three_Appoint(fdf,fcol_list,x_value,item1, item2, item3)
        elif per_sw == 1:
            datain = choose_data_three_per(fdf,fcol_list,x_value,item1, item2, item3)
        
        if datain.iloc[:,0].sum() != 0:
    #         display(list(datain.iloc[:,0])) # Don't delete yet
            chart1 = make_barplot(datain)
            titlestr = f"Monthly Appointments \n with Appointment Status: {col2name(item1)}, HCP Type: {col2name(item2)} & 'Appointment Mode: {col2name(item3)}"
            plt.title(titlestr, fontsize=20)
            plt.show()
        else:
            display('No Data in this combination')


    
    def make_barplot(data1):
        chart1 = sns.barplot(x=data1.index, 
                             y=data1.columns[0], 
                             data=data1,
                             palette='light:b')
        sns.despine(left=True, bottom=False)
        chart1.tick_params(labelrotation=90)
        chart1.set(ylabel= 'Number of Monthly Appointments')
        chart1.set(xlabel= 'Month')
        ax = plt.gca()
        ax.set_yticklabels([])

        t2 = round(max(data1[data1.columns[0]])*1,0)
        l = len(str(t2))
        t1 = 0
        t2 = round(t2*1.2,-(l-1))
        s1 = ((t2-t1)/6)
        listOf_Yticks  = np.arange(t1,t2,s1)
        plt.yticks(listOf_Yticks)
        
        
        if per_sw == 0:
            barplotlabels(chart1)
        elif per_sw == 1:
            barplotlabelsper(chart1)
        return chart1    

    drop_down = widgets.interactive(make_interaction, item1=wlist[0], item2=wlist[1], item3=wlist[2])

    la1 = widgets.Layout(border='solid 2px')

    style = {'description_width': 'initial'}
    drop_down.children[0].description = fcol_list[0].replace('_',' ').title()
    drop_down.children[0].style = style
    drop_down.children[0].value = 'All'

    drop_down.children[1].description = fcol_list[1].replace('_',' ').title()
    drop_down.children[1].style = style
    drop_down.children[1].value = 'All'


    drop_down.children[2].description = fcol_list[2].replace('_',' ').title()
    drop_down.children[2].style = style
    drop_down.children[2].value = 'All'


    app1 = widgets.HBox([drop_down.children[0],drop_down.children[1],drop_down.children[2]])
    app2 = widgets.VBox([app1,drop_down.children[-1]], layout=la1)

    display(app2)






In [66]:
# colpal = sns.diverging_palette(220, 20, sep=1, as_cmap=True)

def make_three_dropdown_bar_time_ser_per(fdf,fcol_list,x_value,per_sw=0):
    wlist = make_lists(fdf,fcol_list)


    def make_interaction(item1, item2, item3):
        datain = choose_data_three_per(fdf,fcol_list,x_value,item1, item2, item3)

        if datain.iloc[:,0].sum() != 0:
    #         display(list(datain.iloc[:,0])) # Don't delete yet
            chart1 = make_barplot(datain)
            titlestr = f"Share Monthly Appointments \n with Appointment Status: {col2name(item1)}, HCP Type: {col2name(item2)} & 'Appointment Mode: {col2name(item3)}"
            plt.title(titlestr, fontsize=20)
            plt.show()
        else:
            display('No Data in this combination')

    
    def make_barplot(data1):
        chart1 = sns.barplot(x=data1.index, 
                             y=data1.columns[0], 
                             data=data1,
                             palette='light:b',
                            )
    #     chart1.bar_label(chart1.containers[0])
        sns.despine(left=True, bottom=False)
        chart1.tick_params(labelrotation=90)
        chart1.set(ylabel= 'Share of Number of Monthly Appointments:  ')
        chart1.set(xlabel= 'Month')
        ax = plt.gca()
        ax.set_yticklabels([])

        t2 = round(max(data1[data1.columns[0]])*1,0)
        l = len(str(t2))
        # print(t2,l,round(t2*1.2,-(l-1))) 
        t1 = 0
        t2 = 1.4
        s1 = ((t2-t1)/6)
        # print((t1,t2,s1,l)) 
        # print((t1,t2,s1)) 
        listOf_Yticks  = np.arange(t1,t2,s1)
        plt.yticks(listOf_Yticks)
 
        
        barplotlabelsper(chart1)
    
        

        return chart1    

    drop_down = widgets.interactive(make_interaction, item1=wlist[0], item2=wlist[1], item3=wlist[2])

    la1 = widgets.Layout(border='solid 2px')

    style = {'description_width': 'initial'}
    drop_down.children[0].description = fcol_list[0].replace('_',' ').title()
    drop_down.children[0].style = style
    drop_down.children[0].value = 'All'

    drop_down.children[1].description = fcol_list[1].replace('_',' ').title()
    drop_down.children[1].style = style
    drop_down.children[1].value = 'All'


    drop_down.children[2].description = fcol_list[2].replace('_',' ').title()
    drop_down.children[2].style = style
    drop_down.children[2].value = 'All'


    app1 = widgets.HBox([drop_down.children[0],drop_down.children[1],drop_down.children[2]])
    app2 = widgets.VBox([app1,drop_down.children[-1]],layout=la1)

    display(app2)






In [67]:
def TimeNCAppoit():
    # Make the chart color and size nice
    sns.color_palette("coolwarm", as_cmap=True)
    sns.set(rc={"figure.figsize":(16, 5)}) #width=3, #height=4
    sns.set_style('white')
    
    df = df_ncs.copy()
    x_value = 'appointment_month'
    
    col_list = ['service_setting*','context_type*',	'national_category*']
        
    make_three_dropdown_bar_time_ser(df,col_list,x_value)
    
    # make_three_dropdown_bar_time_ser_per(df,col_list,x_value)
    
    del col_list




In [68]:
def TimeARAppoit():
    # Make the chart colour and size nice
    sns.color_palette("coolwarm", as_cmap=True)
    sns.set(rc={"figure.figsize":(16, 5)}) #width=3, #height=4
    sns.set_style('white')

    #Bring in the right DataFrame 
    df = df_ars.copy()

    # Format the month variable so it is string and shorter
    x_value = 'appointment_month'
    df[x_value] = df [x_value].astype(str).apply(lambda x: x[0:7])

    # Bring in the right columns to investigate
    col_list = ['appointment_status','hcp_type','appointment_mode','time_between_book_and_appointment']
    col_list = ['appointment_status','hcp_type','appointment_mode']
    col_list = ['appointment_status','hcp_type','appointment_mode*']

    # Execute the functions that create charts 
    make_three_dropdown_bar_time_ser(df,col_list,x_value)
    make_three_dropdown_bar_time_ser_per(df,col_list,x_value)

    # Delete potentially problematic variables
    del col_list
    del df

In [69]:
def TimeARAppoit2():
    # Make the chart colour and size nice
    sns.set(rc={"figure.figsize":(16, 5)}) #width=3, #height=4
    sns.set_style('white')

    #Bring in the right DataFrame 
    df = df_ar.copy()

    # Format the month variable so it is string and shorter
    x_value = 'appointment_month'
    df[x_value] = df [x_value].astype(str).apply(lambda x: x[0:7])

    # Bring in the right columns to investigate
    col_list = ['Region_Full_Name','hcp_type','time_between_book_and_appointment']
    
    # Execute the functions that create charts 
    make_three_dropdown_bar_time_ser(df,col_list,x_value)
    make_three_dropdown_bar_time_ser_per(df,col_list,x_value)

    # Delete potentially problematic variables
    del col_list
    del df

In [70]:
def Make_Delplot_AR_1var():
    sns.set(rc={"figure.figsize":(16, 5)})
    sns.set_style('white')
    df = df_ar.copy()
    plt.ion()
    wlist = list(df.columns)
    wlist.remove('count_of_appointments')
    to_drop_list = ['region_ons_code', 'region_short_code','icb_ons_code', 'short_code', 'ICB_Full_Name',  'delay_atot_float', 'appointment_month', 'ICB_Name']
    wlist = saferemove(wlist, to_drop_list)    
    # print(wlist)
    for i in range(len(wlist)): wlist[i] = col2name(wlist[i])
    
    xlist1 = df[name2col(wlist[0])].unique()
    def make_interaction(item1, item3):
        item1 = name2col(item1)
        # item2 = name2col(item2)
        item3 = name2col(item3)
        print(item3)
        plt.ioff()
        start_time = time.perf_counter()
        xlist1 = df[item1].unique()

        # This if, then tango is needed to handle the complex structure of the muliple widgets
        check1 = 'drop_down' in locals()
        check21 = True
        if check1:
            drop_down.children[1].options = xlist1
            drop_down.children[1].rows=min(10,len(xlist1))  
            check21 = list(drop_down.children[1].value) == list(item3)
        if check21:
            dataout = get_twoway_df(df, 
                                    item1,
                                    fexclude1 = list(item3))
            dataout = get_twoway_data_avdel(dataout, 
                                            fheading=item1,
                                            fexclude1=list(item3),
                                            ftype='sum',
                                            fincol='delay_atot_float')
            chart1 = make_sumbarplot_local(dataout,
                                            fheading=item1,
                                            fexclude1=list(item3))
            total=dataout.delay_atot_float.sum()/dataout.count_of_appointments.sum()
            barplotlabels4(chart1,total)
            chart1.set(ylabel= 'Average Time from Booking to Appointment')
            chart1.set(xlabel= col2name(item1))
            titlestr = f"Average Delay to Appointment, by: {col2name(item1)}"
            plt.title(titlestr, fontsize=20)
            # chart1.set(title= titlestr)
            plt.ion()
            plt.show()
            plt.ioff()
    
    def make_sumbarplot_local(fdata4chart1,fheading, fhuein=[], fexclude1=[], fexclude2=[]):
        sort_data_4chart(fdata4chart1, fheading,'ave_delay','mean')
        chart1 = sns.barplot(x=fheading, 
                                  y='ave_delay', 
                                  data=fdata4chart1,
    #                               order=li,
    #                               hue = fhuein,
                                  estimator='mean',
                                  palette='coolwarm')
        chart1.tick_params(labelrotation=90)
        sns.despine(left=True, bottom=False)        
        return chart1      

    # ----------------------------------------------------------------------------------------------------------------------
    # Global code:
    # Create the two widgets restricting two data series & then the interactive widget and initialisation 
    ddwig1 = widgets.SelectMultiple(options=xlist1,
                                   rows=min(10,len(xlist1)),
                                   description='Variables1',
                                   disabled=False) 

    drop_down = widgets.interactive(make_interaction, 
                                     item1=wlist, 
                                     item3=ddwig1)

    drop_down.children[0].value = wlist[0]
    drop_down.children[0].description = 'Category'
    drop_down.children[1].description = 'To Exclude:'
    drop_down.children[1].options = xlist1 

    ddwig1.layout.width = '250px'

    # box_layout1 = widgets.Layout(justify_content='center')
    # box_layout2 = widgets.Layout(justify_content='center')
    # box_layout3 = widgets.Layout(justify_content='flex-start')

    # app1 = widgets.HBox([drop_down.children[0]], layout=box_layout1)
    # app2 = widgets.VBox([app1, drop_down.children[-1]],layout=box_layout2)
    # app3 = widgets.VBox([drop_down.children[1]],layout=box_layout3)
    # app4 = widgets.HBox([app2,app3])

    # display(app4)

    box_layout1 = widgets.Layout(justify_content='center')
    box_layout2 = widgets.Layout(justify_content='center')
    box_layout3 = widgets.Layout(justify_content='flex-start')
    la1 = widgets.Layout(border='solid 2px')

    app1 = widgets.HBox([drop_down.children[0]], layout=box_layout1)
    app2 = widgets.HBox([drop_down.children[1]] ,layout=box_layout1)
    app3 = widgets.HBox([app1,app2])
    app4 = widgets.VBox([drop_down.children[-1]],layout=box_layout3)
    app5 = widgets.VBox([app3,app4], layout=la1)

    display(app5)





In [71]:
def Make_Durplot_AD_1var():
    sns.set(rc={"figure.figsize":(16, 5)})
    sns.set_style('white')
    df = df_ad.copy()
    plt.ion()
    wlist = list(df.columns)
    wlist.remove('count_of_appointments')
    to_drop_list = ['region_ons_code', 'region_short_code','icb_ons_code', 'short_code',  'delay_atot_float','dur_num', 
                    'dur_num_float', 'dur_tot_float', 'appointment_month', 'ICB_Name', 'sub_icb_location_code',
                    'sub_icb_location_ons_code', 'actual_duration', 'appointment_date']
    wlist = saferemove(wlist, to_drop_list)    
    # print(wlist)
    for i in range(len(wlist)): wlist[i] = col2name(wlist[i])
    xlist1 = df[name2col(wlist[1])].unique()

    def make_interaction(item1,item2, item3):
        plt.ioff()
        item1 = name2col(item1)
        item2 = name2col(item2)
        item3 = name2col(item3)
        start_time = time.perf_counter()
        xlist1 = df[item2].unique()
        check11 = item1 != item2
        if check11:
# This if, then tango, is needed to handle the complex structure of the multiple widgets
            check1 = 'drop_down' in locals()
            check21 = True
            # print(check11)
            if check1 and check11:
                drop_down.children[1].options =  wlist
                drop_down.children[2].options =  xlist1
                # drop_down.children[1].value = wlist[-1]
                drop_down.children[1].rows=min(10,len(xlist1))  
                check21 = list(drop_down.children[2].value) == list(item3)
            if check21:
                dataout = get_twoway_df(df, 
                                        item1,
                                        item2,
                                        fexclude1 = list(item3))
                dataout = get_twoway_data_avdel(dataout, 
                                                fheading=item1,
                                                fhuein=item2,
                                                fexclude2=list(item3),
                                                ftype='sum',
                                                fincol='dur_tot_float')
                chart1 = make_sumbarplot_local(dataout,
                                                fheading=item1,
                                                # fhuein=item2,
                                                fexclude2=list(item3))
                total=dataout['dur_tot_float'].sum()/dataout.count_of_appointments.sum()
                barplotlabels4(chart1,total)
                chart1.set(ylabel= 'Average Time from Booking to Appointment')
                chart1.set(xlabel= col2name(item1))
                titlestr = f"Average Duration of Appointment, by: {col2name(item1)}"
                plt.title(titlestr, fontsize=20)
                plt.ion()
                plt.show()
                plt.ioff()
    
    def make_sumbarplot_local(fdata4chart1,fheading, fhuein=[], fexclude1=[], fexclude2=[]):
        sort_data_4chart(fdata4chart1, fheading,'ave_delay','mean')
        chart1 = sns.barplot(x=fheading, 
                                  y='ave_delay', 
                                  data=fdata4chart1,
    #                               order=li,
                                  # hue = fhuein,
                                  estimator='mean',
                                  palette='coolwarm')
        chart1.tick_params(labelrotation=90)
        sns.set_palette
        sns.despine(left=True, bottom=False)        

        return chart1      

# ----------------------------------------------------------------------------------------------------------------------
# Global code:
# Create the two widgets restricting two data series & then the interactive widget and intialisation 
    ddwig1 = widgets.SelectMultiple(options=xlist1,
                                   rows=min(10,len(xlist1)),
                                   description='Variables1',
                                   disabled=False) 

    drop_down = widgets.interactive(make_interaction, 
                                     item1=wlist, 
                                     item2=wlist, 
                                     item3=ddwig1)

    style = {'description_width': 'initial'}
    drop_down.children[0].value = wlist[0]
    drop_down.children[1].value = wlist[1]
    # drop_down.children[1].options = wlist.remove(wlist[0])
    drop_down.children[2].options = xlist1 
    drop_down.children[0].description = 'Category'
    drop_down.children[1].description = 'Category to Exclude:'
    drop_down.children[2].description = 'Aspect Exclude:'
    drop_down.children[0].style = style
    drop_down.children[1].style = style
    drop_down.children[2].style = style

    ddwig1.layout.width = '250px'

    box_layout1 = widgets.Layout(justify_content='center')
    box_layout2 = widgets.Layout(justify_content='center')
    box_layout3 = widgets.Layout(justify_content='flex-start')
    la1 = widgets.Layout(border='solid 2px')

    app1 = widgets.HBox([drop_down.children[0],drop_down.children[1]], layout=box_layout1)
    app2 = widgets.HBox([drop_down.children[2]] ,layout=box_layout1)
    app3 = widgets.HBox([app1,app2])
    app4 = widgets.VBox([drop_down.children[-1]],layout=box_layout3)
    app5 = widgets.VBox([app3,app4], layout=la1)
    display(app5)
    
    # print (time.perf_counter() - start_time, "seconds")


In [72]:
def Make_Delplot_AR_1var2():
    sns.set(rc={"figure.figsize":(16, 5)})
    sns.set_style('white')
    df = df_ars.copy()
    plt.ion()
    wlist = list(df.columns)
    # print(wlist)
    to_drop_list = ['region_ons_code', 'region_short_code','icb_ons_code', 'short_code',  'delay_atot_float','dur_num', 
                    'dur_num_float', 'dur_tot_float', 'appointment_month', 'ICB_Name', 'sub_icb_location_code',
                    'sub_icb_location_ons_code', 'actual_duration', 'appointment_date','count_of_appointments','appointment_mode',]
    wlist = saferemove(wlist, to_drop_list)    
    # print(wlist)
    for i in range(len(wlist)): wlist[i] = col2name(wlist[i])
    xlist1 = df[name2col(wlist[1])].unique()

    def make_interaction(item1,item2, item3):
        plt.ioff()
        item1 = name2col(item1)
        item2 = name2col(item2)
        item3 = name2col(item3)
        start_time = time.perf_counter()
        xlist1 = df[item2].unique()
        check11 = item1 != item2
        if check11:
# This if, then tango, is needed to handle the complex structure of the multiple widgets
            check1 = 'drop_down' in locals()
            check21 = True
            if check1 and check11:
                drop_down.children[1].options =  wlist
                drop_down.children[2].options =  xlist1
                drop_down.children[2].rows=min(10,len(xlist1))  
                check21 = list(drop_down.children[2].value) == list(item3)
            if check21:
                dataout = get_twoway_df(df, 
                                        item1,
                                        item2,
                                        fexclude1 = list(item3))
                dataout = get_twoway_data_avdel(dataout, 
                                                fheading=item1,
                                                fhuein=item2,
                                                fexclude2=list(item3),
                                                ftype='sum',
                                                fincol='delay_atot_float')
                chart1 = make_sumbarplot_local(dataout,
                                                fheading=item1,
                                                # fhuein=item2,
                                                fexclude2=list(item3))
                total=dataout['delay_atot_float'].sum()/dataout.count_of_appointments.sum()
                barplotlabels4(chart1,total)
                chart1.set(ylabel= 'Average Time from Booking to Appointment')
                chart1.set(xlabel= col2name(item1))
                titlestr = f"Average Delay to Appointment, by: {col2name(item1)}"
                plt.title(titlestr, fontsize=20)
                # chart1.set(title= titlestr)
                plt.ion()
                plt.show()
                plt.ioff()
    
    def make_sumbarplot_local(fdata4chart1,fheading, fhuein=[], fexclude1=[], fexclude2=[]):
        sort_data_4chart(fdata4chart1, fheading,'ave_delay','mean')
        chart1 = sns.barplot(x=fheading, 
                                  y='ave_delay', 
                                  data=fdata4chart1,
    #                               order=li,
                                  # hue = fhuein,
                                  estimator='mean',
                                  palette='coolwarm')
        chart1.tick_params(labelrotation=90)
        sns.set_palette
        sns.despine(left=True, bottom=False)        

        return chart1      

# ----------------------------------------------------------------------------------------------------------------------
# Global code:
# Create the two widgets restricting two data series & then the interactive widget and intialisation 
    ddwig1 = widgets.SelectMultiple(options=xlist1,
                                   rows=min(10,len(xlist1)),
                                   description='Variables1',
                                   disabled=False) 

    drop_down = widgets.interactive(make_interaction, 
                                     item1=wlist, 
                                     item2=wlist, 
                                     item3=ddwig1)

    style = {'description_width': 'initial'}
    drop_down.children[0].value = wlist[0]
    drop_down.children[1].value = wlist[1]
    # drop_down.children[1].options = wlist.remove(wlist[0])
    drop_down.children[2].options = xlist1 
    drop_down.children[0].description = 'Category'
    drop_down.children[1].description = 'Category to Exclude:'
    drop_down.children[2].description = 'Aspect Exclude:'
    drop_down.children[0].style = style
    drop_down.children[1].style = style
    drop_down.children[2].style = style

    ddwig1.layout.width = '250px'

    box_layout1 = widgets.Layout(justify_content='center')
    box_layout2 = widgets.Layout(justify_content='center')
    box_layout3 = widgets.Layout(justify_content='flex-start')
    la1 = widgets.Layout(border='solid 2px')

    app1 = widgets.HBox([drop_down.children[0],drop_down.children[1]], layout=box_layout1)
    app2 = widgets.HBox([drop_down.children[2]] ,layout=box_layout1)
    app3 = widgets.HBox([app1,app2])
    app4 = widgets.VBox([drop_down.children[-1]],layout=box_layout3)
    app5 = widgets.VBox([app3,app4], layout=la1)
    display(app5)
    
    # print (time.perf_counter() - start_time, "seconds")


In [73]:
def StackedTimeSeriesNC(fdf,fmonthloc):
    df = fdf.copy()
    start_time = time.perf_counter()
    sns.color_palette("coolwarm", as_cmap=True)
    # sns.set(rc={"figure.figsize":(16, 5)}) #width=3, #height=4
    # sns.set_style('white')
       
    
        
    plt.ion()
    xlist1 = []
    xlist2 = []

    wlist = list(df.columns)
    to_drop_list = ['region_ons_code', 'region_short_code', 'icb_ons_code', 'short_code', 'ICB_Full_Name', 'count_of_appointments', 'delay_atot_float']
    to_drop_list = to_drop_list + ['sub_icb_location_ons_code', 'sub_icb_short_code',  'appointment_date']
    wlist = saferemove(wlist, to_drop_list)
    # print(wlist)

    for i in range(len(wlist)):
        wlist[i] = col2name(wlist[i])
    # print(wlist)

    
    def make_interaction(item1, item2, item3, item4):
        if item1 != item2:
            item1 = name2col(item1)
            item2 = name2col(item2)
            start_time = time.perf_counter()
            xlist1 = df[item1].unique()
            xlist2 = df[item2].unique()
            # print(xlist1, xlist2)
    
            
    
    # This if, then tango is needed to handle the complex structure of the muliple widgets
    
            check1 = 'drop_down' in locals()
    
            if check1:
                drop_down.children[2].options = xlist1
                drop_down.children[2].rows=min(10,len(xlist1))  
                drop_down.children[3].options = xlist2
                drop_down.children[3].rows=min(10,len(xlist2))  
    
                dataout = get_twoway_df(df, item1,item2,list(item3),list(item4))
    
                check21 = list(drop_down.children[2].value) == list(item3)
                check22 = list(drop_down.children[3].value) == list(item4)
    
                # print('err check1:',drop_down.children[2].value, list(item3))
                # print('err check2:',drop_down.children[3].value, list(item4))
    
    
                if check21 and check22:
                    # Info(dataout)
                    chart1 = make_sumbarplot_local(dataout,item1,item2,list(item3),list(item4))
                    total1 = dataout.count_of_appointments.sum()
                    total2 = dataout.groupby(item1)['count_of_appointments'].sum()
                    barplotlabels2per(chart1,total1,total2)
                    plt.ion()
                    plt.show()
                    print ('1', time.perf_counter() - start_time, "seconds")
    
    
    def make_sumbarplot_local(fdata4chart1, heading, huein, exclude1, exclude2):
        # print(exclude1, exclude2)
    
        fdata4chart1 = fdata4chart1.groupby([heading,huein])['count_of_appointments'].agg('sum').reset_index()
        tempdata = fdata4chart1.groupby(heading)['count_of_appointments'].agg('sum').reset_index()
        chart1 = sns.histplot(data=fdata4chart1, 
                          x=heading,
                          multiple= 'stack', 
                          hue=huein,
    #                       hueorder=li,
                          weights='count_of_appointments', 
                          shrink=0.8,
                          palette='coolwarm')
    
        chart1.tick_params(labelrotation=90)
        sns.move_legend(chart1, "lower center", bbox_to_anchor=(.5, 1.1), ncol=8, title=None, frameon=False,)    
        sns.despine(left=True, bottom=True)        
        chart1.set(ylabel= 'Number and Share of Appointments')
        chart1.set(xlabel= 'Month')
        ax = plt.gca()
        ax.set_yticklabels([])
        titlestr = f"Monthly Appointments \n (Choose which items to exclude)"
        plt.title(titlestr, fontsize=20)

        t1 = 0
        t2 = 44000000
        s1 = ((t2-t1)/6)
        listOf_Yticks  = np.arange(t1,t2,s1)
        plt.yticks(listOf_Yticks)
 
        
        return chart1
                    
                    
    
    ddwig1 = widgets.SelectMultiple(options=xlist1,
                                   rows=min(10,len(xlist1)),
                                   description='Variables1',
                                   disabled=False) 
    
    ddwig2 = widgets.SelectMultiple(options=xlist2,
                                   rows=min(10,len(xlist2)),
                                   description='Category to Exclude',
                                   disabled=False) 
    
    
    drop_down = widgets.interactive(make_interaction, 
                                     item1=wlist, 
                                     item2=wlist,
                                     item3=ddwig1,
                                     item4=ddwig2)
    la1 = widgets.Layout(border='solid 2px')
    
    style = {'description_width': 'initial'}
    drop_down.children[0].value = wlist[fmonthloc]
    drop_down.children[1].description = 'Category'
    drop_down.children[1].value = wlist[0]
    drop_down.children[2].value = ()
    drop_down.children[3].value = ()
    drop_down.children[1].style = style
    drop_down.children[3].style = style

    
    ddwig1.layout.width = '300px'
    ddwig2.layout.width = '300px'
    
    box_layout1 = widgets.Layout(justify_content='center')
    box_layout2 = widgets.Layout(justify_content='center')
    box_layout3 = widgets.Layout(justify_content='flex-start')

    app1 = widgets.HBox([drop_down.children[1]], layout=box_layout1)
    app2 = widgets.HBox([drop_down.children[3]] ,layout=box_layout1)
    app3 = widgets.HBox([app1,app2])
    app4 = widgets.VBox([drop_down.children[-1]],layout=box_layout3)
    app5 = widgets.VBox([app3,app4], layout=la1)

    display(app5)




# monthloc for ncs is -5
# monthloc for ars is 2
    
print (2, time.perf_counter() - start_time, "seconds")



2 2.3024973749998026 seconds


#### Scatter Plot

In [74]:
# Main Scatter Plot
start_time2 = time.perf_counter()
plt.ion()
for _var in to_delete:
    if _var in locals() or _var in globals(): exec(f'del {_var}')

sns.set(rc={"figure.figsize":(16, 5)})
sns.set_style('white')


def TestScatter(fdf):
    df= fdf.reset_index().copy()
    wlist = list(df.columns)
    wlist.remove('index') 
    wlist.remove('icb_ons_code')       
    for i in range(len(wlist)):
        wlist[i] = col2name(wlist[i])


    
    def make_interaction5564(item1,item2,item3):
        item1 = name2col(item1)
        item2 = name2col(item2)
        plt.ioff()
        plt.clf()

        datain = df[[item1,item2]+['icb_ons_code']]        
#         display(datain)                     
        make_scatter_local(datain,item1,item2)

    def make_scatter_local(fdatain,fxdata,fydata,fhue=None):


        chart1 = sns.lmplot(x=fxdata,
                            y=fydata,
                            data=fdatain,
                            # hue=fhue,
                            legend=True,
                            aspect = 2,
                            fit_reg = True,
                            palette = 'light:b'
                            # scatter_kws={'color': colors},
                           )
        fdatain['Location'] = fdatain['icb_ons_code'].map(code2nam_dict_MM)
        
        chart1.set(ylabel= col2name(fydata))
        chart1.set(xlabel= col2name(fxdata))
 
        vx = fdatain[fxdata]
        vy = fdatain[fydata]
        
        vx = sm.add_constant(vx)
        model = sm.OLS(vy,vx).fit()
        fitted = list(model.fittedvalues.values)

        # print(model.summary())
        fitted = pd.DataFrame(fitted)
        fdatain = pd.concat((fdatain, fitted), axis=1)
        fdatain.rename(columns={fdatain.columns[-1]:'yhat'}, inplace=True)
        fdatain['error'] = fdatain[fydata] - fdatain['yhat']
        errorlist = list(fdatain['error'].sort_values(ascending=False))
        # print(errorlist)
        red_fdatain = fdatain[fdatain['error'] >= errorlist[5]]

        # ax = chart1.axes[0, 0]

        
        t1 = round(min(fdatain[fydata])*.8,0)
        t2 = round(max(fdatain[fydata])*1.2,0)
        s1 = round(((t2-t1)/6)*2)/2
        
        # print((t1,t2,s1)) 
        listOf_Yticks  = np.arange(t1,t2,s1)
        plt.yticks(listOf_Yticks)


      
        make_scatter_labels(red_fdatain,fxdata,fydata,'Location')

        format_chart1(chart1)
        r, p = sp.stats.pearsonr(fdatain[fxdata],fdatain[fydata])
        ax = plt.gca()
        bbox1 = {'boxstyle': 'round,pad=1', 'facecolor':'none', 'edgecolor':'maroon'}
        ax.text(.05, 0.1, 'r={:.2f}, p={:.2f}%'.format(r, p*100), transform=ax.transAxes, bbox=bbox1)
        plt.ion()
        plt.show()

        red_fdatain['Region'] = red_fdatain['icb_ons_code'].map(code2nam_dict_MR)
        red_fdatain =  red_fdatain[['Location','Region',fxdata, fydata,'yhat', 'error']]
        red_fdatain.rename(columns={'yhat': 'Predicted '+red_fdatain.columns[-3]}, inplace=True)
        red_fdatain = red_fdatain.sort_values(by='error', axis=0,ascending=False)
  
        display(red_fdatain) 
    
    drop_down3 = widgets.interactive(make_interaction5564, 
                                     item1=wlist,    #This switch is right
                                     item2=wlist,
                                     item3=['Region', 'ICB', 'subICB'] )
    drop_down3.children[0].value = str(wlist[1])
    drop_down3.children[1].value = str(wlist[0])

    box_layout1 = widgets.Layout(justify_content='center')

    app_layout = widgets.Layout(display='flex', flex_flow='row nowrap',align_items='center',border='none',width='100%',margin='5px 5px 5px 5px')
                            
                                
    app11 = widgets.HBox([drop_down3.children[1],
                          drop_down3.children[0],
                          # drop_down3.children[2],
                         ],layout=box_layout1)
    app22 = widgets.VBox([app11,
                          drop_down3.children[-1]], layout=box_layout1)


    display(app22)

# print ('end:',time.perf_counter() - start_time2, "seconds")


# make_scatter_local(df)    

#### Rough scatter

In [75]:
# Rough Scatter Code

start_time2 = time.perf_counter()
plt.ion()
for _var in to_delete:
    if _var in locals() or _var in globals(): exec(f'del {_var}')

def RoughTest():
    df = df_arnc_mod.reset_index().copy()
    wlist, wdict = makelistfromtupels(list(df.columns))
    y_ax_data = df.columns[wdict['monthly_values w/ mon_ave_delay']]
    x_ax_data =  df.columns[wdict['appointment_month']]

    def make_interaction5564(item1):
        plt.ioff()
        plt.clf()
        xdata = df.columns[wdict[item1]]
        display(drop_down3.children[-2])

        datain = df[[
            df.columns[wdict['icb_ons_code']],
            df.columns[wdict['appointment_month']],
            df.columns[wdict['Region']],
            df.columns[wdict['monthly_values w/ mon_ave_delay']],
            df.columns[wdict[item1]]]]
        datain.columns = ['locI','dateI','Region','delay','xaxis']   
        datain = datain.dropna()
        plt.ion()

        make_scatter_local(datain,'xaxis',"delay")
        make_scatter_local_Region(datain,'xaxis',"delay",fhue='Region')
        plt.ion()
        plt.show()

        
    def make_scatter_local(fdatain,fxdata,fydata,fhue=None):

        chart1 = sns.lmplot(x=fxdata,
                           y=fydata,
                           aspect=2,
                           data=fdatain,
                           palette = "light:b"
                           )
        
        format_chart1(chart1)
        # Info(fdatain)
        r, p = sp.stats.pearsonr(fdatain[fxdata],fdatain[fydata])
        ax = plt.gca()
        ax.text(.05, 1.2, 'r={:.2f}, p={:.2f}%'.format(r, p*100),
                transform=ax.transAxes)

    
    def make_scatter_local_Region(fdatain,fxdata,fydata,fhue=None):
        chart1 = sns.lmplot(x=fxdata,
                           y=fydata,
                           aspect=1.8,
                           hue=fhue,
                           data=fdatain,
                           palette = "coolwarm"
                           )         
        format_chart1(chart1)
        # Info(fdatain)
        r, p = sp.stats.pearsonr(fdatain[fxdata],fdatain[fydata])
        ax = plt.gca()
        ax.text(.05, 1.2, 'r={:.2f}, p={:.2f}%'.format(r, p*100),
                transform=ax.transAxes)


        
    drop_down3 = widgets.interactive(make_interaction5564, item1=wlist)
    drop_down3.children[0].value = str(wlist[-2])


    # app22 = widgets.VBox([drop_down.children[0]])

    box_layout1 = widgets.Layout(justify_content='center')

    app22 = widgets.VBox([drop_down3.children[0],
                          drop_down3.children[-1],
#                           drop_down3.children[-2]
                           ],layout=box_layout1)


    display(app22)
#     display(drop_down3)

# print ('end:',time.perf_counter() - start_time2, "seconds")



# Outputs:

## Appointments Regional

### Time Series Analysis of Appointment Status, Appointment Mode

In [76]:
TimeARAppoit()

### Time Series Analysis of Region, HCP Type and Time Between Book And Appointment

In [77]:
TimeARAppoit2()

## National Categories

### Analysis of Time Series Data of National Categories

### Time Series Analysis of Service Setting, Context Type & National Category

In [78]:
TimeNCAppoit()

In [79]:
StackedTimeSeriesNC(df_ncs,-5)

## Measures of Strain

### Tools for Analysis of Time between Booking and Appointment

In [80]:
Make_Delplot_AR_1var()

In [81]:
DurationPlay()

### Tools for Analysis of Appointment Time

In [82]:
Make_Durplot_AD_1var()

## Univariate Geographic Analysis

In [83]:
MapTest()

## Bivariate Analysis 

In [84]:
TestScatter(df_mapM)

## Appendix

### Scatterplot Tool 1

Scatter plot for analysing relationship between different aspects of number of appointments and average delay. It combines monthly data with ICB level data. Bottom chart does separate lines of best fit using NHS regions.

In [85]:
RoughTest()

In [86]:
TestScatter(df_mapMs)